# Import
该文档通过生成仿真数据来研究DRT数据性值

In [170]:
import os
import re
import sys 
from datetime import datetime
from loguru import logger

import matplotlib.pyplot as plt
%matplotlib qt

import torch
import numpy as np
import math

from scipy.linalg import eig, svd, solve
# import scipy

from cvxopt import matrix, solvers
solvers.options['show_progress'] = False


import pyDRTtools as drt



# Definition

## Loewner

In [171]:

def Loewner_Framework(f, Z, REALFLAG = True):
    '''==================================================
        Construct Loewner Pencel
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        ==================================================
    '''
    _n = len(f)
    s = 2j * np.pi * f

    # Ensuring the input have an even number of elements 
    # for constructing the model having real entries
    if REALFLAG:
        if _n % 2 != 0:
            _n = _n - 1

    # Left & Right Data for Loewner Framework
    s_left  = s[:_n:2]
    H_left  = Z[:_n:2]
    s_right = s[1:_n:2] 
    H_right = Z[1:_n:2]

    # Construct complex conjugate values for ensuring model having real entries
    if REALFLAG:
        s_left  = np.stack([s_left, s_left.conj()], axis=1).flatten()
        H_left  = np.stack([H_left, H_left.conj()], axis=1).flatten()
        s_right = np.stack([s_right, s_right.conj()], axis=1).flatten()
        H_right = np.stack([H_right, H_right.conj()], axis=1).flatten()

    # Constructing the Loewner Matrix & Shifted Loewner Matrix
    # L   = (H_left[:,None] - H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    # Ls  = (s_left[:,None] * H_left[:,None] - s_right[None,:] * H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    L   = (H_left[None,:] - H_right[:,None]) / (s_left[None,:] - s_right[:,None])
    Ls  = (s_left[None,:] * H_left[None,:] - s_right[:,None] * H_right[:,None]) / (s_left[None,:] - s_right[:,None])

    # Transforming the conplex L & Ls to obtain matrices with real entries
    if REALFLAG:
        _J_diag = np.eye(_n//2)
        _J  = (1/np.sqrt(2)) * np.array([[1, 1j], [1, -1j]])
        _J  = np.kron(_J_diag, _J)

        L       = (_J.conj().T @ L @ _J).real
        Ls      = (_J.conj().T @ Ls @ _J).real
        H_left  = ((_J.T @ H_left).T).real
        H_right = (_J.conj().T @ H_right).real

        
    return L, Ls, H_left, H_right

def state_space_model(L, Ls, H_left, H_right):
    '''==================================================
        Construct state space model from Loewner Pencel
        Parameter: 
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        Returen:
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        ==================================================
    '''
    # rank of the Loewner Pencel
    _rank = np.linalg.matrix_rank(np.concatenate((L, Ls), axis=0))
    Y_L, svd_L, X_L = svd(L, full_matrices=False, lapack_driver='gesvd')
    X_L = X_L.T
    
    # Reduced state space model interpolating the data
    Yk = Y_L[:, :_rank]
    Xk = X_L[:, :_rank]

    Ek = -Yk.T@L@Xk
    Ak = -Yk.T@Ls@Xk
    Bk = Yk.T@H_right
    Ck = H_left.T@Xk

    return Ek, Ak, Bk, Ck

def DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG = True, real_th = 1e5):
    '''==================================================
        Transform state space model to DRT model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        Returen:
            R_i:    R_i from RC pair in DRT
            C_i:    C_i from RC pair in DRT
            tau_i   tau_i from RC pair in DRT
        ==================================================
    '''
    # Solve Av= λEv & wT A= λ wT E & Res = CvwB/wEv, wEv =  δ
    _pol, _U = eig(Ak, Ek)     # 
    wB = solve(_U, solve(Ek,Bk))
    Cv = Ck @ _U
    _res = Cv * wB

    # Calculate R_i & tau_i
    tau_i   = (-1/_pol) 
    R_i     = (-_res / _pol)
    C_i     = (1/_res)
    # tau_i   = abs(-1/_pol) 

    if REALFLAG:
        # real_ratio = np.where(np.abs(tau_i.imag) == 0, np.inf, np.abs(tau_i.real / (tau_i.imag+1e-20)))
        # tau_i = np.abs(tau_i[real_ratio > real_th])
        # R_i = np.abs(R_i[real_ratio > real_th])
        # C_i = np.abs(C_i[real_ratio > real_th])


        real_ratio = np.where(np.abs(tau_i.imag) == 0, np.inf, np.abs(tau_i.real / (tau_i.imag+1e-20)))       
        real_mask = (real_ratio > real_th) & (tau_i.real>0) & (R_i.real>0)
        tau_i = tau_i[real_mask].real
        R_i = R_i[real_mask].real
        C_i = tau_i/R_i


        # real_mask = (tau_i.real>0)
        # tau_i = np.real(tau_i[real_mask])
        # R_i = np.real(R_i[real_mask])

        # R_i = R_i[tau_i.argsort()]
        # tau_i = tau_i[tau_i.argsort()]
        
        # uniq_cnt_mask = np.abs(np.diff(tau_i, prepend=0)) < 1e-16
        # uniq_mask = np.abs(np.diff(tau_i, append=0)) > 1e-16
        
        # R_i[uniq_cnt_mask] = R_i[uniq_cnt_mask] * 2
        # R_i = R_i[uniq_mask]
        # tau_i = tau_i[uniq_mask]
        # C_i = tau_i / R_i
        

    return R_i, C_i, tau_i

def DRT_Reconstruction_SSM(Ek, Ak, Bk, Ck, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            R_i:    R_i from RC pair in DRT
            tau_i   tau_i from RC pair in DRT
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f
    H = np.array([Ck @ solve(si * Ek - Ak, Bk) for si in s])
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ


def DRT_Reconstruction_DRT(R_i, tau_i, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f  # Broadcasting tau_i to match f
    _RC = R_i[None, :] / (1+s[:,None] * tau_i[None,:])
    H = np.sum(_RC, axis=1)

    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ

def Loe_singularity_analysis(f, Z, REALFLAG = True):
    '''==================================================
        DRT Singularity Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Y, svd_L, X = svd(np.concatenate([L, Ls]), full_matrices=False)

    return svd_L

def Loe_Analysis_Single(f, Z, REALFLAG = True):
    '''==================================================
        DRT Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
            H:          reconstructed impedance values
            res_ReZ:    relative error of real part of impedance values
            res_ImZ:    relative error of imaginary part of impedance values
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Ek, Ak, Bk, Ck = state_space_model(L, Ls, H_left, H_right)
    R_i, C_i, tau_i = DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG=True)
    # H, res_ReZ, res_ImZ = DRT_Reconstruction_SSM(Ek, Ak, Bk, Ck, f, Z)
    H, res_ReZ, res_ImZ = DRT_Reconstruction_DRT(R_i, tau_i, f, Z)

    return R_i, C_i, tau_i, H, res_ReZ, res_ImZ
    
def Loe_Analysis_Batch(chData, REALFLAG = True):
    '''==================================================
        DRT Analysis for Batch Data
        Parameter: 
            chData:     list of tuples (f, Z) for each channel
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            results:    list of tuples (R_i, C_i, tau_i, H, res_ReZ, res_ImZ) for each channel
        ==================================================
    '''
    DRTdata = []
    f = chData[0,0,:]
    for i in range(chData.shape[0]):
        _Z = chData[i,1,:] + 1j*chData[i,2,:]
        R_i, C_i, tau_i, _, _, _ = DRT_Analysis_Single(f, _Z, REALFLAG)
        DRTdata.append((np.array([R_i, C_i, tau_i])))
    return DRTdata



## Tikhonov

In [172]:
def TikDRT(f, Z, RLC_Flag=[False, False, False], custom_lambda = None):
    '''==================================================
        Tikhonov DRT Deconvolution
        Parameter: 
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
            ch_eis: 3 x n_freq Matrix: [freq, Real, Imag]
        Returen:
            tau_vec: time domain vector
            x: DRT result
            n_extend: number of extend RLC parameters
        ==================================================
    '''
    ## Freq domain data prepare
    
    n_freq = len(f)
    # freq_vec = f
    # Z_exp = Z
    freq_vec = np.flip(f)
    Z_exp = np.flip(Z)

    '''Hyper Parameters'''
    # Time domain parameters
    log_tau_min = np.log10(1/(2*np.pi*freq_vec[0]))  
    log_tau_max = np.log10(1/(2*np.pi*freq_vec[-1])) 
    # log_tau_min = np.log10(1/(freq_vec[0]))  
    # log_tau_max = np.log10(1/(freq_vec[-1]))   
    n_tau = n_freq

    # tau_vec = 1/(2*np.pi*freq_vec)
    # tau_vec = 1/(2*np.pi*100*freq_vec)
    # tau_vec = np.logspace(-4,4, n_tau, endpoint=True)
    tau_vec = np.logspace(log_tau_min, log_tau_max, num = n_tau, endpoint=True)

    # log_tau = np.log(tau_vec)
    # tau_vec = np.logspace(-6, 0, n_tau, endpoint=True)
    # freq_vec = np.flip(np.logspace(0, 6, n_freq, endpoint=True))
    


    # Discretization matrices Parameters
    # Use RBF Kernel to initialize the A matrix
    RBF_shape_control = 'FWHM Coefficient' 
    RBF_coeff = 0.5
    # RBF_type = 'Piecewise Linear'
    RBF_type = 'Gaussian'
    # RBF_type = 'C0 Matern'
    # RBF_type = 'C2 Matern'
    # RBF_type = 'C4 Matern'
    # RBF_type = 'C6 Matern'
    # RBF_type = 'Inverse Quadratic'

    # Cross-validation Method for optimize lambda (Tikhonov regularization parameter) 
    cv_type = 'GCV'     # Generalized Cross Validation
    # cv_type = 'mGCV'    # Modified Generalized Cross Validation
    # cv_type = 'rGCV'    # Robust Generalized Cross Validation
    # cv_type = 'LC'      # L-curve
    # cv_type = 're-im'   # Real-Imaginary discrepancy
    # cv_type = 'kf'      # k-fold cross-validation

    '''Compute the RBF Shape Parameter Epsilon'''
    epsilon = drt.basics.compute_epsilon(freq_vec, RBF_coeff, RBF_type, RBF_shape_control)

    # logger.info(f"{epsilon}")
    # return
    '''Compute the discretization matrices'''
    A_re = drt.basics.assemble_A_re(freq_vec, tau_vec, epsilon, RBF_type)
    n_extend = np.sum(RLC_Flag)   
    if RLC_Flag[2]:
        A_re_C_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_C_0, A_re)) 
    if RLC_Flag[1]:
        A_re_L_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_L_0, A_re))
    if RLC_Flag[0]:
        A_re_R_inf  = np.ones((n_freq, 1))
        A_re        = np.hstack((A_re_R_inf, A_re)) 

    A_im = drt.basics.assemble_A_im(freq_vec, tau_vec, epsilon, RBF_type)
    if RLC_Flag[2]:
        A_im_C_0    = -1/(2*np.pi*freq_vec.reshape(-1,1))
        A_im        = np.hstack((A_im_C_0, A_im))
    if RLC_Flag[1]:
        A_im_L_0    = 2*np.pi*freq_vec.reshape(-1,1)
        A_im        = np.hstack((A_im_L_0, A_im))
    if RLC_Flag[0]:
        A_im_R_inf  = np.zeros((n_freq, 1)) 
        A_im        = np.hstack((A_im_R_inf, A_im))

    A = np.vstack((A_re, A_im))
     


    '''Compute the differentiation matrices for Tiknonov regularization'''
    M2 = np.zeros((n_tau+n_extend, n_tau+n_extend))
    M2[n_extend:,n_extend:] = drt.basics.assemble_M_2(tau_vec, epsilon, RBF_type)

    '''Optimize lambda'''
    if custom_lambda is None:
        log_lambda_init = -3 # ln(lambda_init = 0.001)
        # lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", RLC_Flag[1], log_lambda_init, cv_type)
        lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", 0, log_lambda_init, cv_type)
    else: 
        lambda_opt = custom_lambda
    # logger.info(f"Lambda: {lambda_opt}")
    '''Deconvolve The DRT from the EIS Data'''
    # Set Bound Constraints
    # lb = np.zeros([n_tau+n_extend])
    # bound_mat = np.eye(lb.shape[0])
    H_combined, c_combined = drt.basics.quad_format_combined(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, lambda_opt)
    G = matrix(-np.identity(Z_exp.shape[0]+n_extend))
    h = matrix(np.zeros(Z_exp.shape[0]+n_extend))


    # Deconvolved DRT - Old
    sol = solvers.qp(matrix(H_combined), matrix(c_combined), G, h)
    x = np.array(sol['x'])

    # Deconvolved DRT - New
    # x_var = cp.Variable(H_combined.shape[0])
    # objective = cp.Minimize(0.5 * cp.quad_form(x_var, H_combined) + c_combined.T @ x_var)
    # constraints = [x_var >= 0]
    # prob = cp.Problem(objective, constraints)
    # prob.solve(solver=cp.ECOS, verbose=False,
    #        abstol=1e-6, reltol=1e-6)

    # x = x_var.value

    # Output layer
    H = A@x
    H = H[:n_freq] + 1j*H[n_freq:]
    H = np.flip(H).flatten()
    R_i = x[n_extend:].flatten() / 2
    C_i = tau_vec/R_i

    return R_i, C_i, tau_vec, H, x[:n_extend].flatten(), lambda_opt



def Tik_Analysis_Batch(chData, RLC_Flag=[False, False, False], custom_lambda = None):
    '''==================================================
        DRT Analysis for Batch Data
        Parameter: 
            chData:     list of tuples (f, Z) for each channel
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            results:    list of tuples (R_i, C_i, tau_i, H, res_ReZ, res_ImZ) for each channel
        ==================================================
    '''
    DRTdata = []
    H_list = []
    f = chData[0,0,:]
    for i in range(chData.shape[0]):
        _Z = chData[i,1,:] + 1j*chData[i,2,:]
        R_i, C_i, tau_i, H, _, _= TikDRT(f, _Z, RLC_Flag, custom_lambda)
        DRTdata.append((np.array([R_i, C_i, tau_i])))
        H_list.append(H)
    return DRTdata, H_list

def Tik_Res(Z, H):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return res_ReZ, res_ImZ



## Plot

In [173]:

def DRT_Plot_Batch(DRTdata):

    
    fig = plt.figure()
    axis1 = fig.add_subplot(121)
    axis2 = fig.add_subplot(122)


    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(DRTdata)):
        ch_drt = DRTdata[i]
        _color = cmap(i/len(DRTdata))

        axis1.plot(ch_drt[2,:], ch_drt[0,:], color=_color, linewidth=2, alpha=0.5)
        axis2.plot(ch_drt[2,:], 1/ch_drt[1,:], color=_color, linewidth=2, alpha=0.5)

    axis1.set_xscale('log')
    axis1.set_yscale('log')
    axis2.set_xscale('log')
    axis2.set_yscale('log')


# Simulation

## Monte-Carlo

### Data Simulation

In [185]:
ECM_TYPE = 'R-(R||C)-(R||C)'
# ECM_TYPE = 'Randle'
# ECM_TYPE = 'R-((R-Q)||Q)'
# ECM_TYPE = 'R-((R-Q)||Q)-5000'


if ECM_TYPE == 'R-(R||C)-(R||C)':
    # Element
    R1 = 200; # Ω
    R2 = 100; # Ω
    R0 = 70;  # Ω

    C1 = 2.5e-3; # 
    C2 = 1e-4;   # 

    # tau1 = R1*C1;  # s
    # tau2 = R2*C2;  # s

    f = np.logspace(-2,2,41);  # Hz

    # Calculation of the impedance dataset 
    Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'Randle':
    R0 = 70

    R1 = 10000 
    C1 = 2.5e-9

    Y1 = 1e-5 
    
    n1 = 0.66
    f = np.logspace(-1,5,61);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Z_sim = np.array([ R0 + (R1+Q1(w))/(1+1j*w*C1*(R1+Q1(w))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag
        
elif ECM_TYPE == 'R-((R-Q)||Q)':
    R0 = 70

    R1 = 50000 
    
    Y1 = 1e-8
    n1 = 0.8
    
    Y2 = 1e-5 
    n2 = 0.66

    f = np.logspace(-1,5,61);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'R-((R-Q)||Q)-5000':
    R0 = 70

    R1 = 50000 
    
    Y1 = 1e-8
    n1 = 0.8
    
    Y2 = 1e-5 
    n2 = 0.66

    f = np.logspace(-1,5,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag



plt.figure()
plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')


### Run

In [194]:
if f.shape[0] < 1000:
    # R_i, C_i, tau_i, H, res_ReZ, res_ImZ = Loe_Analysis_Single(f, Z_sim, REALFLAG=True)
    # svd_L = Loe_singularity_analysis(f, Z_sim, REALFLAG=True)

    # plt.figure()
    # plt.loglog(tau_i,R_i, '.')


    _n_noise = 100
    _drt_noise = []

    Z_org = Z_sim
    # Z_org = Z_sim_noise

    fig = plt.figure()
    axis = fig.add_subplot(111)



    for i in range(_n_noise):
        _Z_noise = Z_org + np.random.normal(0, 0.0001, Z_org.shape) * Z_org.real + np.random.normal(0, 0.0001, Z_org.shape) * Z_org.imag
        
        _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(f, _Z_noise, REALFLAG=True)
        _drt_noise.append(np.array([_R_i, _C_i, _tau_i]))
        # axis.scatter(_tau_i[1:-1],_R_i[1:-1], s=1, color='blue')
        # axis.scatter(_tau_i[1:-1],_R_i[1:-1]*_R_i.shape[0], s=1,color='red')
        plt.loglog(_tau_i,_R_i, '.',color='gray')
        plt.loglog(_tau_i,_R_i*_R_i.shape[0], '.',color='gray')


        
    _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(f, Z_org, REALFLAG=True)
    _drt_noise.append(np.array([_R_i, _C_i, _tau_i]))
    # axis.scatter(_tau_i,_R_i, color='red')
    # axis.scatter(_tau_i,_R_i*R_i.shape[0], color='red')

    axis.set_xscale('log')
    axis.set_yscale('log')

## Bootstrap

### Data Simulation

In [218]:
# ECM_TYPE = 'R-(R||C)-(R||C)'
# ECM_TYPE = 'R-((R-C))||C)'
# ECM_TYPE = 'Randle'
# ECM_TYPE = 'R-((R-Q2)||Q1)-5000'
# ECM_TYPE = 'R-(Q1 || (R-W))-5000'
# ECM_TYPE = 'C-R-(Q1 || (R-W))-5000'
ECM_TYPE = '((R-(C1||W)) || C2)-5000'
# ECM_TYPE = 'C-R-W'
# ECM_TYPE = 'C-R-Q'
# ECM_TYPE = 'C-R-(W||C1)'
# ECM_TYPE = 'C||(R-(W||C1))'
# ECM_TYPE = '(R-(R||Q))||R||Q)'


if ECM_TYPE == 'R-(R||C)-(R||C)':
    # Element
    R1 = 200; # Ω
    R2 = 100; # Ω
    R0 = 70;  # Ω

    C1 = 2.5e-3; # 
    C2 = 1e-4;   # 

    # tau1 = R1*C1;  # s
    # tau2 = R2*C2;  # s

    f = np.logspace(0,8,5000);  # Hz

    # Calculation of the impedance dataset 
    Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'R-((R-C))||C)':
    # Element
    R1 = 20000; # Ω
    R0 = 100;  # Ω

    C1 = 2.5e-3; # 
    C2 = 1e-4;   # 

    # tau1 = R1*C1;  # s
    # tau2 = R2*C2;  # s

    f = np.logspace(-2,2,5000);  # Hz

    Q1 = lambda x: 1/(C1*1j*x)
    Q2 = lambda x: 1/(C2*1j*x)

    # Calculation of the impedance dataset 
    # Z_sim = np.array([1/(1/Q1(w) + 1/(R1+Q2(w))) +R0 for w in 2*np.pi*f])
    Z_sim = np.array([1/(1/Q1(w) + 1/(R1+Q2(w))) for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag



elif ECM_TYPE == 'R-((R-Q2)||Q1)-5000':
    R0 = 1000

    R1 = 10000
    
    Y1 = 1e-9
    n1 = 1
    
    Y2 = 1e-5
    n2 = 0.6

    f = np.logspace(0,6,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag


elif ECM_TYPE == '((R-(C1||W)) || C2)-5000':
    R0 = 1000

    C1 = 1e-8
    C2 = 1e-10

    
    WR = 1e5
    WT = 1e-3
    n2 = 0.6

    f = np.logspace(0,8,5000);  # Hz 

    W1 = lambda x: WR/((WT*1j*x)**n2)
    Q1 = lambda x: 1/(C1*(1j*x))
    Q2 = lambda x: 1/(C2*(1j*x))

    Z_sim = np.array([ 1/( 1/Q2(w) + 1/( R0+ 1/(1/Q1(w)+1/W1(w)) ) ) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag


elif ECM_TYPE == 'R-(Q1 || (R-W))-5000':
    R0 = 1000

    R1 = 1e5
    
    Y1 = 1e-10
    n1 = 0.9
    
    WR = 100
    WT = 1e-7
    n2 = 0.6

    f = np.logspace(-5,6,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: WR/((1j*x*WT)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag


elif ECM_TYPE == 'C-R-(Q1 || (R-W))-5000':
    R0 = 1000
    C0 = 1e-7

    R1 = 1e5
    
    Y1 = 1e-10
    n1 = 0.9
    
    WR = 100
    WT = 1e-7
    n2 = 0.6

    f = np.logspace(0,6,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: WR/((1j*x*WT)**n2)

    Z_sim = np.array([ Q0(w)+R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag
    Z_sim_noise = Z_sim + np.random.normal(0, 1e-2, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 1e-2, Z_sim.shape) * Z_sim.imag


elif ECM_TYPE == 'C-R-W':    
    R0 = 1000
    C0 = 1e-7

    
    WR = 100
    WT = 1e-1
    n1 = 0.5


    WR2 = 10000
    WT2 = 1e-3
    n2 = 0.1

    f = np.logspace(0,6,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: WR/((1j*x*WT)**n1)
    Q2 = lambda x: WR/((1j*x*WT2)**n2)

    # Z_sim = np.array([ Q0(w)+R0 +Q1(w) +Q2(w)  for w in 2*np.pi*f])
    Z_sim = np.array([ R0 +Q1(w) +Q2(w)  for w in 2*np.pi*f])
    # Z_sim = np.array([ Q0(w)+R0 +Q1(w)  for w in 2*np.pi*f])
    # Z_sim = np.array([ Q0(w)+R0   for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'C-R-Q':    
    R0 = 1000
    C0 = 1e-7

    Y1 = 1e-10
    n1 = 0.9


    Y2 = 1e-5
    n2 = 0.1

    f = np.logspace(0,6,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    # Z_sim = np.array([ Q0(w)+R0 +Q1(w) +Q2(w)  for w in 2*np.pi*f])
    Z_sim = np.array([ Q0(w)+R0 +Q1(w)  for w in 2*np.pi*f])
    # Z_sim = np.array([ Q0(w)+R0   for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'C-R-(W||C1)':    
    R0 = 1000
    C0 = 1e-7
    C1 = 1e-10
    
    WR = 1000000
    WT = 1e-1
    n1 = 0.1


    f = np.logspace(0,6,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: 1/((1j*x*C1))
    Q2 = lambda x: WR/((1j*x*WT)**n1)

    Z_sim = np.array([ Q0(w)+R0 +(1/(1/Q1(w)+1/Q2(w))) for w in 2*np.pi*f])
    # Z_sim = np.array([ Q0(w)+R0 +Q1(w)  for w in 2*np.pi*f])
    # Z_sim = np.array([ Q0(w)+R0   for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'C||(R-(W||C1))':    
    R0 = 1000
    C0 = 9e-11
    C1 = 7e-9
    
    WR = 1e3
    WT = 1e-2
    n1 = 0.30


    f = np.logspace(0,8,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: 1/((1j*x*C1))
    Q2 = lambda x: WR/((1j*x*WT)**n1)

    Z_sim = np.array([ 1/(1/Q0(w) + 1/(R0 + 1/(1/Q1(w)+1/Q2(w)))) for w in 2*np.pi*f])

    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag


elif ECM_TYPE == '(R-(R||Q))||R||Q)':    
    R0 = 1000
    R1 = 1e4
    R2 = 1e6

    Y1 = 1e-10
    n1 = 0.9


    Y2 = 1e-11
    n2 = 0.9

    f = np.logspace(0,6,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    # Z_sim = np.array([ Q0(w)+R0 +Q1(w) +Q2(w)  for w in 2*np.pi*f])
    Z_sim = np.array([ 1/(1/(R0+1/(1/R1+1/Q1(w)) + 1/R2 + 1/Q2(w)  ))  for w in 2*np.pi*f])
    # Z_sim = np.array([ Q0(w)+R0   for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag





plt.figure()
plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')
# plt.loglog(f, np.abs(Z_sim), label='Ampletude')
# plt.semilogx(f, np.angle(Z_sim)*180/np.pi, label='Ampletude')
# plt.plot((1/Z_sim).real, (1/Z_sim).imag, label='Simulated Data')
plt.title(ECM_TYPE)


Text(0.5, 1.0, '((R-(C1||W)) || C2)-5000')

In [219]:
# f = np.array([1, 1e40])
# # Z_sim = np.array([ 1/( 1/Q2(w) + 1/( R0+ 1/(1/Q1(w)+1/W1(w)) ) ) for w in 2*np.pi*f])
# # Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
# # Z_sim = np.array([ Q0(w)+R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
# pc = 1/Z_sim[0].imag/(2*np.pi*f[0])
# pc

# _C_nf

### Run

In [220]:
def stratified_subsample(total_size, n_subsample, idx_base=1000, seed=None):
    """
    将 total_size 个点均匀分成 n_subsample 段，每段随机取 1 个索引，无放回采样。
    """
    if seed is not None:
        np.random.seed(seed)
    
    indices = np.arange(idx_base, total_size)
    bins = np.array_split(indices, n_subsample)
    sampled_indices = [np.random.choice(bin, 1)[0] for bin in bins]
    
    return np.array(sampled_indices)

In [221]:

# Z_org = Z_sim
Z_org = Z_sim_noise

_f = f
_Z = Z_org



n_batch = 100
n_point = 100

_f_bootstrap = []
_Z_bootstrap = []

_Z_bootstrap_nf = []
for i in range(n_batch):
    _idx = stratified_subsample(_f.shape[0], n_point, idx_base=1000)
    # _idx[0] = 1000
    # _idx[-1] = _f.shape[0]-1

    _f_bootstrap.append(_f[_idx])
    _Z_bootstrap.append(_Z[_idx])
    _Z_bootstrap_nf.append(Z_sim[_idx])

_f = np.stack(_f_bootstrap, axis=0)
_Z = np.stack(_Z_bootstrap, axis=0)
_Z_nf = np.stack(_Z_bootstrap_nf, axis=0)

if False:
    plt.figure()
    for i in range(_f.shape[0]):
        plt.loglog(_f[i,:], np.abs(_Z[i,:]))


In [222]:
_drt_list = []
_nf_list = []

fig = plt.figure()
axis = fig.add_subplot(131)
axis2 = fig.add_subplot(132)
axis3 = fig.add_subplot(133)
for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _R_nf, _C_nf, _tau_nf, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z_nf[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))
    _nf_list.append(np.array([_R_nf, _C_nf, _tau_nf]))

    # R_0 = _R_i[0]
    # C_0 = _C_i[-1]
    # Z_cal = _Z[i,:] - R_0 - 1/(2j*np.pi*_f[i,:]*C_0)
    # _R_cal, _C_cal, _tau_cal, _, _, _ = Loe_Analysis_Single(_f[i,:], Z_cal, REALFLAG=True)



    # axis.scatter(_tau_i,_R_i, s=2, color = 'gray')
    # axis.scatter(_R_i, _C_i, s=2, color = 'gray')
    # axis.scatter(_tau_i[0],_R_i[0], s=2)
    # axis.scatter(_tau_i[0],_R_i[0]*_R_i.shape[0], s=2)

    ## Last Point Discuss
    # axis.scatter(_tau_i[1:-4],_R_i[1:-4], s=2, color = 'gray')
    # axis.scatter(_tau_i[0],_R_i[0], s=2, color = 'red')
    # axis.scatter(_tau_i[-4],_R_i[-4], s=2, color='blue')
    # axis.scatter(_tau_i[-3],_R_i[-3], s=2, color='green')
    # axis.scatter(_tau_i[-2],_R_i[-2], s=2, color='orange')
    # axis.scatter(_tau_i[-1],_R_i[-1], s=2, color='red')

    ## Order Correction Discuss - Conclusion: correction should NOT be applied here
    # axis.scatter(_tau_i[1:-10],_R_i[1:-10], s=2, color='red')
    # axis.scatter(_tau_i[1:-10],_R_i[1:-10]*_R_i.shape[0], s=2, color='blue')

    # R or C
    axis.scatter(_R_i[1:]*_C_i[1:], _R_i[1:], s=2, color = 'gray')
    axis.scatter(_R_i[0]*_C_i[0], _R_i[0], s=2, color = 'orange')
    axis.scatter(_R_nf[:]*_C_nf[:], _R_nf[:], s=10, color = 'blue')
    # axis.vlines(1/2/np.pi/f[0], np.min(_R_nf[:]), np.max(_R_nf[:]))
    # axis.vlines(1/2/np.pi/f[-1], np.min(_R_nf[:]), np.max(_R_nf[:]))


    # axis.scatter(_R_i[1:]*_C_i[1:], 1/_C_i[1:], s=2, color = 'gray')
    # axis.scatter(_R_i[0]*_C_i[0], 1/_C_i[0], s=2, color = 'orange')
    # axis.scatter(_R_nf[:]*_C_nf[:], 1/_C_nf[:], s=2, color = 'blue')
    # axis.vlines(1/2/np.pi/f[0], np.min(1/_C_nf[:]), np.max(1/_C_nf[:]))
    # axis.vlines(1/2/np.pi/f[-1], np.min(1/_C_nf[:]), np.max(1/_C_nf[:]))



    # ## RC Discussion
    axis2.scatter(_R_i[1:-1], _C_i[1:-1], s=2, color = 'gray')
    axis2.scatter(_R_i[-1], _C_i[-1], s=2, color = 'red')
    axis2.scatter(_R_i[0], _C_i[0], s=2, color = 'orange')
    axis2.scatter(_R_nf[:], _C_nf[:], s=10, color = 'blue')


    ## R/C Discussion
    # axis.scatter(_R_i[1:-1]*_C_i[1:-1], _R_i[1:-1]/_C_i[1:-1], s=2, color = 'gray')
    # axis.scatter(_R_i[-1]*_C_i[-1], _R_i[-1]/_C_i[-1], s=2, color = 'red')
    # axis.scatter(_R_i[0]*_C_i[0], _R_i[0]/_C_i[0], s=2, color = 'orange')
    # axis.scatter(_R_nf[:]*_C_nf[:], _R_nf[:]/_C_nf[:], s=2, color = 'blue')
    

    ## R/C/(x+1/x)
    _tt = _R_i[:]*_C_i[:]
    _yy = _R_i[:] / _C_i[:]
    # _yy = _yy / (((_tt/1e0)**-1) + ((_tt/1e0)**1))
    # _yy = _yy / (((_tt/1e-4)**-1) + ((_tt/1e-4)**1))
    _yy = _yy / (((_tt/1e6)**-1) + ((_tt/1e-14)**1))
    axis3.scatter(_tt[1:-1], _yy[1:-1], s=2, color = 'gray')
    axis3.scatter(_tt[-1], _yy[-1], s=2, color = 'red')
    axis3.scatter(_tt[0], _yy[0], s=2, color = 'orange')



    _tt = _R_nf[:]*_C_nf[:]
    _yy = _R_nf[:] / _C_nf[:]
    # _yy = _yy / (((_tt/1e0)**-1) + ((_tt/1e0)**1))
    # _yy = _yy / (((_tt/1e-4)**-1) + ((_tt/1e-4)**1))
    # _yy = _yy / (((_tt/_R_nf[0]/_R_nf[0])**-1) + ((_tt/_C_nf[-1]/_C_nf[-1])**1))
    _yy = _yy / (((_tt/1e6)**-1) + ((_tt/1e-14)**1))
    axis3.scatter(_tt[1:-1], _yy[1:-1], s=2, color = 'blue')
    axis3.scatter(_tt[-1], _yy[-1], s=2, color = 'blue')
    axis3.scatter(_tt[0], _yy[0], s=2, color = 'blue')


    # axis.scatter(_tt, (((_tt/1e-4)**-1) + ((_tt/1e-4)**1)), s=2, color = 'orange')
    # axis.scatter(_tt, (((_tt/1e-2)**-1) + ((_tt/1e-6)**1)), s=2, color = 'red')
    # axis.scatter(_tt, (((_tt/1e-6)**-1) + ((_tt/1e-2)**1)), s=2, color = 'red')
    


# axis.set_xlim([1e-10,1e2])
# axis.set_ylim([1e0,1e10])
axis.set_xscale('log')
axis.set_yscale('log')
axis.set_aspect('equal')

axis2.set_xscale('log')
axis2.set_yscale('log')
axis2.set_aspect('equal')

axis3.set_xscale('log')
axis3.set_yscale('log')
axis3.set_aspect('equal')


C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\129456308.py:102: LinAlgWarning: Ill-conditioned matrix (rcond=5.59352e-17): result may not be accurate.
  wB = solve(_U, solve(Ek,Bk))
C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\129456308.py:102: LinAlgWarning: Ill-conditioned matrix (rcond=2.14387e-17): result may not be accurate.
  wB = solve(_U, solve(Ek,Bk))
C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\129456308.py:102: LinAlgWarning: Ill-conditioned matrix (rcond=2.72576e-17): result may not be accurate.
  wB = solve(_U, solve(Ek,Bk))
C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\129456308.py:102: LinAlgWarning: Ill-conditioned matrix (rcond=3.20031e-17): result may not be accurate.
  wB = solve(_U, solve(Ek,Bk))
C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\129456308.py:102: LinAlgWarning: Ill-conditioned matrix (rcond=4.76851e-17): result may not be accurate.
  wB = solve(_U, solve(Ek,Bk))
C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\12945630

In [223]:

_R_list = np.array([i[0,0] for i in _drt_list])
_R_para = 1/np.sum(1/_R_list)
np.mean(_R_list)


np.float64(692.7023368268835)

In [224]:
print(_C_i[-1])
print(_R_i[0])

4.105692229683039e-08
982.5200505394247


## Theoretical CPE 

### Data Simulation

In [369]:
ECM_TYPE = 'R-(R||Q)'

if  ECM_TYPE == 'R-(R||Q)':    
    R0 = 0
    C0 = 1e-7

    
    # R1 = 10000
    # Y1 = 5e-8
    # n1 = 0.9

    
    R1 = 10000
    Y1 = 1e-6
    n1 = 0.5


    f = np.logspace(0,6,5000);  # Hz 

    Q0 = lambda x: 1/((1j*x*C0))
    Q1 = lambda x: 1/(Y1*(1j*x)**n1)


    noise_level = 1e-3
    Z_sim = np.array([ R0 + (1/(1/R1 + 1/Q1(w)))  for w in 2*np.pi*f])
    # Z_sim = np.array([ R0 + Q0(w) + (1/(1/R1 + 1/Q1(w)))  for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.imag



if True:
    plt.figure()
    plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')
    # plt.plot((1/Z_sim).real, (1/Z_sim).imag, label='Simulated Data')
    plt.title(ECM_TYPE)

In [364]:
def stratified_subsample(total_size, n_subsample, idx_base=1000, seed=None):
    """
    将 total_size 个点均匀分成 n_subsample 段，每段随机取 1 个索引，无放回采样。
    """
    if seed is not None:
        np.random.seed(seed)
    
    indices = np.arange(idx_base, total_size)
    bins = np.array_split(indices, n_subsample)
    sampled_indices = [np.random.choice(bin, 1)[0] for bin in bins]
    
    return np.array(sampled_indices)

# Z_org = Z_sim
Z_org = Z_sim_noise

_f = f
_Z = Z_org



n_batch = 100
n_point = 100

_f_bootstrap = []
_Z_bootstrap = []

_Z_bootstrap_nf = []
for i in range(n_batch):
    # _idx = stratified_subsample(_f.shape[0], n_point, idx_base=1000)
    _idx = stratified_subsample(_f.shape[0], n_point, idx_base=0)
    # _idx[0] = 1000
    # _idx[-1] = _f.shape[0]-1

    _f_bootstrap.append(_f[_idx])
    _Z_bootstrap.append(_Z[_idx])
    _Z_bootstrap_nf.append(Z_sim[_idx])

_f = np.stack(_f_bootstrap, axis=0)
_Z = np.stack(_Z_bootstrap, axis=0)
_Z_nf = np.stack(_Z_bootstrap_nf, axis=0)

if False:
    plt.figure()
    for i in range(_f.shape[0]):
        plt.loglog(_f[i,:], np.abs(_Z[i,:]))


### From PDF to Data


尝试了很多方法，包括用quad算G积分，用G不定积分解析式F计算G积分，均匀采样或者不均匀采样，都可以！

In [51]:
from scipy.integrate import quad

def log_uniform_weights_from_F(xi, F):
    """
    xi: log10均匀采样点 (e.g. np.logspace(-3, 3, 100))
    F: 核函数 G(x) 的不定积分函数 (callable)
    return: 每个 xi 对应的积分区间的权重 yi
    """
    log_xi = np.log10(xi)
    delta_log = log_xi[1] - log_xi[0]  # 等间距
    log_edges = np.concatenate([
        [log_xi[0] - delta_log / 2],
        (log_xi[:-1] + log_xi[1:]) / 2,
        [log_xi[-1] + delta_log / 2]
    ])
    x_edges = 10**log_edges
    yi = F(x_edges[1:]) - F(x_edges[:-1])
    return yi



def nonuniform_weights_from_Fx(xi, F):
    """
    xi: 非等距采样点，必须单调递增
    F: 不定积分函数
    return: 每个 xi 的积分权重 yi
    """
    xi = np.log(xi)
    x_edges = np.zeros(len(xi)+1)
    x_edges[1:-1] = (xi[:-1] + xi[1:]) / 2
    x_edges[0] = xi[0] - (xi[1] - xi[0]) / 2
    x_edges[-1] = xi[-1] + (xi[-1] - xi[-2]) / 2
    yi = F(x_edges[1:]) - F(x_edges[:-1])
    return yi

def nonuniform_weights_from_Gx(xi, G):
    """
    xi: 非等距采样点，必须单调递增
    F: 不定积分函数
    return: 每个 xi 的积分权重 yi
    """
    xi = np.log(xi)
    x_edges = np.zeros(len(xi)+1)
    x_edges[1:-1] = (xi[:-1] + xi[1:]) / 2
    x_edges[0] = xi[0] - (xi[1] - xi[0]) / 2
    x_edges[-1] = xi[-1] + (xi[-1] - xi[-2]) / 2
    yi = np.array([quad(G, x_edges[i], x_edges[i+1])[0] for i in range(xi.shape[0])])
    return yi



In [ ]:
_drt_list = []
_nf_list = []

_theo_den_list = []
_theo_list = []
_theo_list_G = []
_theo_uni_list = []

_theo_non_drt = []
_theo_uni_drt = []



t0 = np.power(R1*Y1, 1/n1)


CPE_G1 = lambda x: np.sin(n1*np.pi) / (np.cosh(n1*(x-np.log(t0))) + np.cos(n1*np.pi)) /2/np.pi
CPE_G2 = lambda t: np.sin(n1*np.pi) / (np.cosh(n1*(np.log(t/t0))) + np.cos(n1*np.pi)) /2/np.pi
CPE_F1 = lambda x: np.arctan(np.tan(n1*np.pi/2) * np.tanh(n1/2*(x-np.log(t0)))) /np.pi/n1
CPE_F2 = lambda t: np.arctan(np.tan(n1*np.pi/2) * np.tanh(n1/2*(np.log(t/t0)))) /np.pi/n1

uni_order = []
uni_order_nf = []

for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _R_nf, _C_nf, _tau_nf, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z_nf[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))
    _nf_list.append(np.array([_R_nf, _C_nf, _tau_nf]))


    ## nf
    theo_tau_list = _tau_nf[_tau_nf.argsort()]
    D_den_theo = np.array([CPE_G2(i) for i in theo_tau_list]) * R1
    _theo_den_list.append(np.array([theo_tau_list, D_den_theo]))
    

    D_theo = nonuniform_weights_from_Fx(theo_tau_list, CPE_F1) * R1
    _theo_list.append(np.array([theo_tau_list, D_theo]))

    
    D_G_theo = nonuniform_weights_from_Gx(theo_tau_list, CPE_G1) * R1
    _theo_list_G.append(np.array([theo_tau_list, D_G_theo]))

    
    # tau_uni = np.logspace(-np.log10(2*np.pi*f.max()), -np.log10(2*np.pi*f.min()), _tau_nf.shape[0])
    # D_uni = log_uniform_weights_from_F(tau_uni, CPE_F2) * R1
    # _theo_uni_list.append(np.array([tau_uni, D_uni]))

    uni_order_nf.append(_tau_nf.shape[0])

    ## drt_noise
    theo_tau_list = _tau_i[_tau_i.argsort()]

    
    D_theo = nonuniform_weights_from_Fx(theo_tau_list, CPE_F1) * R1
    _theo_non_drt.append(np.array([theo_tau_list, D_theo]))
    
    uni_order.append(_tau_i.shape[0])
    

uni_order = int(np.round(np.mean(uni_order)))
tau_uni = np.logspace(-np.log10(2*np.pi*f.max()), -np.log10(2*np.pi*f.min()), uni_order)
# tau_uni = np.logspace(np.log10(t0/1e2), np.log10(t0*1e2), _tau_i.shape[0])
D_uni = log_uniform_weights_from_F(tau_uni, CPE_F2) * R1
_theo_uni_drt.append(np.array([tau_uni, D_uni]))



uni_order_nf = int(np.round(np.mean(uni_order_nf)))

tau_uni = np.logspace(-np.log10(2*np.pi*f.max()), -np.log10(2*np.pi*f.min()), uni_order_nf)
D_uni = log_uniform_weights_from_F(tau_uni, CPE_F2) * R1
_theo_uni_list.append(np.array([tau_uni, D_uni]))





In [61]:
# D_uni[(tau_uni>_RQ_peak/10) & (tau_uni<_RQ_peak*10)]
# tau_uni[(tau_uni>_RQ_peak/10) & (tau_uni<_RQ_peak*10)]

In [60]:
# log_uniform_weights_from_F(
#     [_RQ_peak/(__alpha/__alpha),_RQ_peak/__alpha,_RQ_peak,_RQ_peak*__alpha,_RQ_peak*__alpha*__alpha], 
#     CPE_F2) * R1

# __Rp = log_uniform_weights_from_F(
#     [_RQ_peak/(__alpha**2),_RQ_peak,_RQ_peak*(__alpha**2)], 
#     CPE_F2) * R1
# __Rp

# log_uniform_weights_from_F(
#     tau_uni[(tau_uni>_RQ_peak/10) & (tau_uni<_RQ_peak*10)], 
#     CPE_F2) * R1
# _RQ_peak

In [62]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

def find_half_max_loglog(x, y, x0, plot=True):
    # 转 log-log
    logx = np.log10(x)
    logy = np.log10(y)

    # 插值函数（单调性允许使用线性插值）
    interp_logy = interp1d(logx, logy, kind='linear', fill_value='extrapolate')

    # 找到log(x0)对应的log(y0)
    logx0 = np.log10(x0)
    logy0 = float(interp_logy(logx0))
    y0 = 10 ** logy0
    logy_half = logy0 - np.log10(2)

    # 寻找左半部分小于logx0的点
    left_mask = logx < logx0
    right_mask = logx > logx0

    def get_crossing_x(logx_sub, logy_sub):
        # 只处理 y 递增到 y0/2 处的交点
        diff = logy_sub - logy_half
        sign = np.sign(diff)
        cross_indices = np.where(np.diff(sign))[0]

        if len(cross_indices) == 0:
            return None, np.inf  # 无法找到交点

        idx = cross_indices[0]
        x1 = np.interp(logy_half, logy_sub[idx:idx+2], logx_sub[idx:idx+2])
        error = abs(diff[idx]) + abs(diff[idx+1])
        return 10 ** x1, error

    # 处理左右
    x1, err1 = get_crossing_x(logx[left_mask][::-1], logy[left_mask][::-1])
    x2, err2 = get_crossing_x(logx[right_mask], logy[right_mask])

    if plot:
        plt.figure(figsize=(7, 5))
        plt.loglog(x, y, 'o-', label='Data')
        plt.axvline(x0, color='gray', linestyle='--', label='x0')
        plt.axhline(y0, color='green', linestyle='--', label='y0')
        plt.axhline(y0/2, color='red', linestyle='--', label='y0/2')
        if x1: plt.axvline(x1, color='red', linestyle=':', label='x1')
        if x2: plt.axvline(x2, color='red', linestyle=':', label='x2')
        plt.legend()
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Half Maximum Detection (log-log)')
        plt.grid(True, which='both', ls='--')
        plt.show()

    return y0, x1, x2, err1, err2





def robust_peak_halfwidth(x, y, x0, window_ratio=0.2, smooth_factor=1e-3, plot=False):
    x = np.asarray(x)
    y = np.asarray(y)
    
    # 1. 在x0附近截取局部数据（自适应窗口）
    x_range = x.max() - x.min()
    window = x_range * window_ratio
    mask = (x > x0 - window) & (x < x0 + window)
    x_local = x[mask]
    y_local = y[mask]

    if len(x_local) < 5:
        raise ValueError("局部点太少，无法拟合")

    # 2. 拟合局部三次多项式，估计 y0
    p = np.polyfit(x_local, y_local, deg=3)
    y0 = np.polyval(p, x0)

    # 3. 全局拟合光滑曲线（spline），忽略明显的outlier
    spline = UnivariateSpline(x, y - y0/2, s=smooth_factor)
    roots = spline.roots()

    # 4. 找最接近x0的左右两个点作为x1, x2
    roots = np.array(roots)
    left = roots[roots < x0]
    right = roots[roots > x0]

    if len(left) == 0 or len(right) == 0:
        raise RuntimeError("无法找到两个半高点")

    x1 = left[-1]
    x2 = right[0]

    # 5. 估算误差：用拟合残差作为粗略标准差
    residuals = y_local - np.polyval(p, x_local)
    error = np.std(residuals)

    # 可视化检查
    if plot:
        plt.scatter(x, y, label='Raw Data', alpha=0.3)
        x_fit = np.linspace(min(x), max(x), 1000)
        plt.plot(x_fit, np.polyval(p, x_fit), 'g--', label='Local Polyfit')
        plt.axvline(x1, color='r', linestyle='--', label='x1, x2')
        plt.axvline(x2, color='r', linestyle='--')
        plt.axhline(y0, color='k', linestyle='--', label='y0')
        plt.axhline(y0/2, color='grey', linestyle=':')
        plt.scatter([x0], [y0], color='blue', label='x0, y0')
        plt.xscale('log')
        plt.yscale('log')
        plt.legend()
        plt.title("Robust Peak Analysis")
        plt.show()

    return y0, x1, x2, error



from statsmodels.nonparametric.smoothers_lowess import lowess

_drt_all = np.concatenate(_drt_list, axis=1)
_drt_all = _drt_all[:,_drt_all[2,:].argsort()]
_RQ_peak = (R1*Y1)**(1/n1)


_drtRC_log_loess = lowess(np.log(_drt_all[0,:]), np.log(_drt_all[2,:]), frac=0.05, it=3, return_sorted=False)
_drtRC_log_loess = np.exp(_drtRC_log_loess)

y0, x1, x2, error1, error2 = find_half_max_loglog(_drt_all[2,:],_drtRC_log_loess, _RQ_peak, plot=True)
axis = plt.gca()
axis.plot(_drt_all[2,:], _drt_all[0,:])
# y0, x1, x2, error = estimate_half_max_width_loglog(_drt_all[2,:],_drt_all[0,:], _RQ_peak, plot=True)
# y0, x1, x2, error = robust_peak_halfwidth(_drt_all[2,:],_drt_all[0,:], _RQ_peak, 
#                                           window_ratio=0.2, smooth_factor=1e-3,plot=True)
logger.info(f"\nPeak[{_RQ_peak}, {y0}]\n [{x1}, {x2}]")
# logger.info(f"\n{_RQ_peak/x1}\t{x2/_RQ_peak}")
__eta = _RQ_peak/x1

2025-08-09 20:55:20.630 | INFO     | __main__:<module>:140 - 
Peak[9.973682993875135e-08, 755.7658339186091]
 [5.50018919939476e-08, 1.9175789132526927e-07]


In [70]:


# __t0 = (R1*Y1)**(1/n1)
# __fp = 6557
# __alpha = np.sqrt(tau_uni_drt[1]/tau_uni_drt[0])
# __tl = 1.296e-4
# __tr = 3.704e-4
# __tr = __t0**2/__tl
# axis[1].vlines(__t0,D_uni_drt.min(), D_uni_drt.max())
# axis[1].vlines(__t0/__alpha,D_uni_drt.min(), D_uni_drt.max())
# axis[1].vlines(__t0*__alpha,D_uni_drt.min(), D_uni_drt.max())
# axis[1].hlines(__fp/2,tau_uni_drt.min(), tau_uni_drt.max())

In [64]:

fig, axis = plt.subplots(1,2)
axis = axis.flatten()
cmap = plt.get_cmap('rainbow_r')
for i in range(_f.shape[0]):
    if i!=10: continue
    _R_i, _C_i, _tau_i = _drt_list[i][0,:], _drt_list[i][1,:], _drt_list[i][2,:]
    _R_nf, _C_nf, _tau_nf = _nf_list[i][0,:], _nf_list[i][1,:], _nf_list[i][2,:]

    theo_den_tau, D_den_theo = _theo_den_list[i][0,:], _theo_den_list[i][1,:]
    theo_tau, D_theo = _theo_list[i][0,:], _theo_list[i][1,:]
    theo_G_tau, D_G_theo = _theo_list_G[i][0,:], _theo_list_G[i][1,:]

    # tau_uni, D_uni = _theo_uni_list[i][0,:], _theo_uni_list[i][1,:]
    tau_uni, D_uni = _theo_uni_list[0][0,:], _theo_uni_list[0][1,:]

    
    tau_non_drt, D_non_drt = _theo_non_drt[i][0,:], _theo_non_drt[i][1,:]

    tau_uni_drt, D_uni_drt = _theo_uni_drt[0][0,:], _theo_uni_drt[0][1,:]

    


    # axis[0].scatter(_R_i[0]*_C_i[0], _R_i[0], s=2, color = 'orange')
    axis[0].scatter(_R_nf[:]*_C_nf[:], _R_nf[:], s=10, color = 'blue')
    axis[0].scatter(theo_den_tau, D_den_theo, s=10, color='green')
    axis[0].scatter(theo_tau, D_theo, s=10, color = 'red')
    # axis[0].scatter(theo_G_tau, D_G_theo, color = 'red')
    axis[0].scatter(tau_uni, D_uni, s=10, color = 'orange')
    axis[0].scatter(_R_i[1:]*_C_i[1:], _R_i[1:]*2, s=2, color = 'gray')

    
    axis[1].scatter(_R_i[1:]*_C_i[1:], _R_i[1:], s=5, color = 'gray')
    # axis[1].scatter(theo_den_tau, D_den_theo, s=2, color='green')
    axis[1].scatter(tau_non_drt, D_non_drt, s=5, color = 'red')
    # axis[1].scatter(tau_uni_drt, D_uni_drt, s=50, color = 'orange')
    

    axis[1].scatter(tau_uni_drt, D_uni_drt, s=50, color = cmap(i/_f.shape[0]))
    # axis[1].scatter(tau_uni, D_uni, s=50, color = 'orange')


axis[0].set_xscale('log')
axis[0].set_yscale('log')
axis[0].set_aspect('equal')
axis[1].set_xscale('log')
axis[1].set_yscale('log')
axis[1].set_aspect('equal')




In [65]:
i = 2
__alpha = np.sqrt(tau_uni_drt[i+1]/tau_uni_drt[i])
__alpha
__eta

np.float64(1.8133345294690293)

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

# 参数设定
a = __alpha
eta = __eta

def lhs_eq1(beta):
    return np.tanh(beta * np.log(a) / np.pi)

def rhs_eq1(beta):
    t1 = np.tanh(beta * np.log(a * eta) / np.pi)
    t2 = np.tanh(beta * np.log(a / eta) / np.pi)
    denom = 1 - (np.tan(beta) ** 2) * t1 * t2
    if np.isclose(denom, 0):
        return np.nan  # 避免除以0
    return (t1 + t2) / denom

# 定义误差函数用于求解 beta
def residual_eq1(beta):
    return lhs_eq1(beta) - rhs_eq1(beta)

# 绘图检查曲线交点
b_vals = np.linspace(1e-6, np.pi / 2 - 1e-6, 500)
lhs_vals = [lhs_eq1(b) for b in b_vals]
rhs_vals = [rhs_eq1(b) for b in b_vals]

plt.figure()
plt.plot(b_vals, lhs_vals, label='LHS', color='blue')
plt.plot(b_vals, rhs_vals, label='RHS', color='red')
plt.xlabel('β')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.title('Equation 1: LHS vs RHS')
plt.show()

# 数值求解：选择合适初值
sol1 = root_scalar(residual_eq1, bracket=[0.01, np.pi/2 - 0.01], method='brentq')
print(f"[Equation 1] 解为 beta ≈ {sol1.root:.6f}\tphi={2*sol1.root/np.pi:.6f}\terror:{(2*sol1.root/np.pi-n1)/n1:.6f}")

# --------------------------------------------

# 另一个公式
def Fp(beta):
    return np.arctan(np.tan(beta) * np.tanh(beta * np.log(a) / np.pi))

def Fh(beta):
    t1 = np.arctan(np.tan(beta) * np.tanh(beta * np.log(a * eta) / np.pi))
    t2 = np.arctan(np.tan(beta) * np.tanh(beta * np.log(a / eta) / np.pi))
    return t1 + t2

def residual_eq2(beta):
    return Fp(beta) - Fh(beta)

# 绘图
Fp_vals = [Fp(b) for b in b_vals]
Fh_vals = [Fh(b) for b in b_vals]
# plt.figure()
plt.plot(b_vals, Fp_vals, label='Fp', color='green')
plt.plot(b_vals, Fh_vals, label='Fh', color='orange')
plt.xlabel('β')
plt.ylabel('Angle')
# plt.xscale('log')
# plt.yscale('log')
plt.legend()
plt.grid(True)
plt.title('Equation 2: Fp vs Fh')
plt.show()

# 数值求解
# sol2 = root_scalar(residual_eq2, bracket=[1e-6, np.pi/2 - 1e-6], method='bisect')
# print(f"[Equation 2] 解为 beta ≈ {sol2.root:.6f}")
__beta = sol1.root

[Equation 1] 解为 beta ≈ 1.194812	phi=0.760641	error:-0.154843


In [68]:
__beta = sol1.root
__fp_1 = 1/__beta*np.arctan(np.tan(__beta)*np.tanh(__beta*np.log(__alpha)/np.pi))
__fp_2 = 1/__beta*(np.arctan(np.tan(__beta)*np.tanh(__beta*np.log(__alpha*__eta)/np.pi))
                 +np.arctan(np.tan(__beta)*np.tanh(__beta*np.log(__alpha/__eta)/np.pi)))
logger.info(f"\n{y0/__fp_1}, {y0/__fp_2}, {y0/np.sqrt(__fp_1*__fp_2)}")
# logger.info(f"\nR1_error: {(y0/__fp_1-R1)/R1}, {(y0-__Rp[1])/__Rp[1]}")
# logger.info(f"\nfp_error: {(__fp_1-__Rp[1]/R1)/(__Rp[1]/R1)}")
# __fp
# __fp_1


2025-08-09 20:56:11.024 | INFO     | __main__:<module>:5 - 
1059.5103670807343, 574.8664714380806, 780.4338448425768


### From Data to IPDF to PDF

#### Definition

In [229]:
import numpy as np
from scipy.optimize import lsq_linear

def _build_L(N, diff_order=2):
    if diff_order == 0:
        L = np.eye(N)
    elif diff_order == 1:
        L = np.eye(N, k=0) - np.eye(N, k=1)
        L = L[:-1]
    elif diff_order == 2:
        L = np.zeros((N-2, N))
        for i in range(N-2):
            L[i, i:i+3] = [1, -2, 1]
    else:
        raise ValueError("diff_order must be 0,1,2")
    return L

def tikhonov_inversion_robust(
    A, y, lam=1e-2, nonneg=True, diff_order=2,
    weights=None,                   # 可传入逐区间权重向量（或与 y 同形）
    huber_delta=None,               # 设为正数启用 Huber IRLS，如 1e-2
    max_irls_iter=25, irls_tol=1e-6,
    dx=None,                        # 标量或长度 N 的数组；用于归一化约束
    norm_mu=1e3,                    # 归一化约束强度；越大越接近硬约束
    verbose=False
):
    """
    Solve: min ||W^{1/2}(A p - y)||^2 + lam ||L p||^2  s.t. (soft) sum(p*dx)=1, p>=0 (optional)
    - A, y: 也可为 list[ndarray]，表示多组 IPDF；函数内部会按行堆叠
    - weights: 与 y 同形或堆叠后长度相同的权重（可与 Huber 同时使用）
    - huber_delta: 启用 Huber IRLS 以抑制离群；None 则不用
    - dx: 标量（等距）或长度 N 的数组（不等距）；若 None 则归一化不施加
    - norm_mu: 归一化软约束的权重（建议 1e2~1e5）
    """
    # ---- 堆叠多组数据 ----
    if isinstance(A, (list, tuple)):
        A_stack = np.vstack(A)
        y_stack = np.concatenate(y)
    else:
        A_stack = np.asarray(A)
        y_stack = np.asarray(y)
    m, N = A_stack.shape
    y_stack = y_stack.reshape(-1)

    # ---- 差分矩阵 ----
    L = _build_L(N, diff_order=diff_order)

    # ---- 初始权重向量 ----
    if weights is None:
        w = np.ones(m)
    else:
        w = np.asarray(weights).reshape(-1)
        if w.size != m:
            raise ValueError("weights length mismatch after stacking")

    # ---- 归一化约束向量 q ----
    if dx is not None:
        if np.isscalar(dx):
            q = np.full(N, float(dx))
        else:
            q = np.asarray(dx).reshape(-1)
            if q.size != N:
                raise ValueError("dx length must equal N (number of columns in A)")
    else:
        q = None

    # ---- IRLS 主循环（若未启用 Huber，则只跑一轮）----
    p = np.full(N, 1.0 / max(N, 1))  # 初值：均匀
    last_obj = np.inf

    for it in range(1 if huber_delta is None else max_irls_iter):
        # 形成加权 A,y
        W_sqrt = np.sqrt(w)
        Aw = (W_sqrt[:, None] * A_stack)
        yw = W_sqrt * y_stack

        # 增广系统
        A_aug = [Aw, np.sqrt(lam) * L]
        y_aug = [yw, np.zeros(L.shape[0])]
        if q is not None and norm_mu > 0:
            A_aug.append(np.sqrt(norm_mu) * q.reshape(1, -1))
            y_aug.append(np.array([np.sqrt(norm_mu) * 1.0]))
        A_aug = np.vstack(A_aug)
        y_aug = np.concatenate(y_aug)

        # 解：非负/无约束
        if nonneg:
            res = lsq_linear(A_aug, y_aug, bounds=(0, np.inf), lsmr_tol='auto', max_iter=5000)
            p = res.x
        else:
            # 正规方程（注意可能更数值不稳，建议优先用 lsq_linear）
            ATA = A_aug.T @ A_aug
            ATy = A_aug.T @ y_aug
            p = np.linalg.solve(ATA, ATy)

        # 可选：求目标函数值以监控收敛
        resid = A_stack @ p - y_stack
        if huber_delta is None:
            # 纯加权二范数
            data_term = np.sum((np.sqrt(w) * resid) ** 2)
        else:
            # Huber 代价（仅用于监控，不影响权重更新）
            a = np.abs(resid)
            delta = huber_delta
            data_term = np.sum(np.where(a <= delta, 0.5 * a**2, delta * (a - 0.5 * delta)) * w)

        reg_term = lam * np.sum((L @ p) ** 2)
        norm_term = 0.0
        if q is not None and norm_mu > 0:
            norm_term = norm_mu * (q @ p - 1.0) ** 2
        obj = data_term + reg_term + norm_term
        if verbose:
            print(f"[IRLS {it+1}] obj={obj:.6e}, data={data_term:.6e}, reg={reg_term:.6e}, norm={norm_term:.6e}")

        # IRLS: 更新权重 w（基于上一轮残差）
        if huber_delta is not None:
            r = resid
            a = np.abs(r)
            delta = huber_delta
            # Huber 的等价权（W）= ψ(r)/r；对应到二次近似中 sqrt(w)
            # 这里直接用 w = max(delta/|r|, 1) 的裁剪版本以避免 0/0
            new_w = np.ones_like(a)
            mask = a > delta
            new_w[mask] = delta / (a[mask] + 1e-12)
            # 累乘用户权重（若提供）
            if weights is not None:
                new_w *= np.asarray(weights).reshape(-1)
            # 收敛判据
            if np.linalg.norm(new_w - w) / (np.linalg.norm(w) + 1e-12) < irls_tol:
                w = new_w
                break
            w = new_w
        else:
            break

    # 最后再做一次严格归一化（避免数值误差）
    if q is not None:
        Z = q @ p
        if Z > 0:
            p = p / Z

    return p


In [230]:
import numpy as np

def build_A_trapz(bin_edges, grid):
    """
    构造梯形积分法的 A 矩阵
    bin_edges: shape (M+1,) 测量区间边界
    grid: shape (N+1,) PDF 网格的边界（cell 边界）
    返回: A shape (M, N+1) ，这里 p_j 定义在节点（grid 顶点）上
    """
    M = len(bin_edges) - 1
    N = len(grid) - 1
    # 节点个数 = N+1
    A = np.zeros((M, N+1))

    # 对每个测量区间
    for i in range(M):
        L, R = bin_edges[i], bin_edges[i+1]
        for j in range(N):
            x0, x1 = grid[j], grid[j+1]

            # 找区间和单元的重叠部分
            left = max(L, x0)
            right = min(R, x1)
            overlap = right - left
            if overlap <= 0:
                continue

            # 梯形积分的权重分配：
            # 单元的左节点和右节点各占一半，但要按重叠比例缩放
            cell_len = x1 - x0
            w_left = overlap / cell_len / 2
            w_right = overlap / cell_len / 2

            A[i, j] += w_left * cell_len
            A[i, j+1] += w_right * cell_len

    return A


In [231]:

from scipy.optimize import lsq_linear
from scipy.linalg import toeplitz

# -----------------------------
# Helpers: build integration matrix A
# -----------------------------
def build_A(bin_edges, grid):
    """
    构造 A 矩阵：每个测量区间（由 bin_edges 给出）对 grid 中每个小单元的积分权重。
    bin_edges: shape (M+1,) 测量区间边界，测量 i 对应 [bin_edges[i], bin_edges[i+1]]
    grid: shape (N+1,) 细分网格的边界（通常可以等于 bin_edges 或更细）
    返回 A shape (M, N) ，p_j 定义在 grid 单元上（中心或cell-average）
    """
    M = len(bin_edges)-1
    N = len(grid)-1
    A = np.zeros((M, N))
    # 对每个测量区间 i，对每个细网格单元 j 计算重叠长度
    for i in range(M):
        L, R = bin_edges[i], bin_edges[i+1]
        # for each cell j
        for j in range(N):
            a, b = grid[j], grid[j+1]
            overlap = max(0.0, min(R, b) - max(L, a))
            A[i, j] = overlap
    return A

# -----------------------------
# Method 1: naive IPDF / Δx
# -----------------------------
def naive_pdf_from_ipdf(ipdf, bin_edges):
    widths = np.diff(bin_edges)
    pdf = ipdf / widths
    grid_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])
    return grid_centers, pdf

# -----------------------------
# Method 2: Tikhonov (regularized LS), optionally non-negative
# -----------------------------
def tikhonov_inversion(A, y, lam=1e-3, nonneg=True, diff_order=2):
    """
    Solve min ||A p - y||^2 + lam ||L p||^2
    If nonneg: enforce p >= 0 via lsq_linear
    diff_order: 0 -> identity, 1 -> first diff, 2 -> second diff
    """
    N = A.shape[1]
    # build L
    if diff_order == 0:
        L = np.eye(N)
    elif diff_order == 1:
        L = np.eye(N, k=0) - np.eye(N, k=1)
        L = L[:-1]  # shape (N-1, N)
    elif diff_order == 2:
        L = np.zeros((N-2, N))
        for i in range(N-2):
            L[i, i] = 1
            L[i, i+1] = -2
            L[i, i+2] = 1
    else:
        raise ValueError("diff_order 0/1/2 only")
    # Augmented system trick: [A; sqrt(lam)*L] p = [y; 0]
    A_aug = np.vstack([A, np.sqrt(lam)*L])
    y_aug = np.concatenate([y, np.zeros(L.shape[0])])
    if nonneg:
        res = lsq_linear(A_aug, y_aug, bounds=(0, np.inf), lsmr_tol='auto', max_iter=2000)
        p = res.x
    else:
        # analytical solution via normal eqn (A^T A + lam L^T L) p = A^T y
        ATA = A.T @ A
        LTL = L.T @ L
        mat = ATA + lam * LTL
        rhs = A.T @ y
        p = np.linalg.solve(mat, rhs)
    return p

##
def Tikhonov_IPDF2PDF(tau, R, lam = 1e-3):
 
    tau_log = np.log10(tau)

    integral_edge = (tau_log[:-1] + tau_log[:-1])/2    
    pdf_grid =  np.linspace(math.floor(integral_edge[0]), math.ceil(integral_edge[-1]), 
                            1+20*(math.ceil(integral_edge[-1]) - math.floor(integral_edge[0])))
    



    # Method 1: naive
    # pdf_x, pdf_est = naive_pdf_from_ipdf(R[1:-1], integral_edge)
    # pdf_x = np.power(10, pdf_x)
    # Method 2: tikhonov ls (nonneg)
    A = build_A(integral_edge, pdf_grid)
    # A = build_A_trapz(integral_edge, (pdf_grid[:-1] + pdf_grid[1:]) / 2)
    pdf_x = np.power(10, (pdf_grid[:-1] + pdf_grid[1:]) / 2)
    pdf_est = tikhonov_inversion(A, R[1:-1], lam=lam, nonneg=True, diff_order=2)
    # pdf_est = tikhonov_inversion_robust(A, R[1:-1], lam=lam, nonneg=True, diff_order=2)

    return pdf_x, pdf_est

    

def Tikhonov_IPDF2PDF_list(drt_list, lam = 1e-3):

    R_list          = [_drt[0,1:-1] for _drt in drt_list]
    tau_list        = [_drt[2,:] for _drt in drt_list]

    R_list_conc     = np.concatenate(R_list, axis=0)
    tau_list_conc   = np.concatenate(tau_list, axis=0)
    tau_list_conc_log = np.log10(tau_list_conc)

    pdf_grid =  np.linspace(math.floor(np.min(tau_list_conc_log)), math.ceil(np.max(tau_list_conc_log)), 
                            1+20*(math.ceil(np.max(tau_list_conc_log)) - math.floor(np.min(tau_list_conc_log))))
        

    A_list = []
    for i in range(len(tau_list)):
        tau_log = np.log10(tau_list[i])
        integral_edge = (tau_log[:-1] + tau_log[:-1])/2    
        A = build_A(integral_edge, pdf_grid)
        # A = build_A_trapz(integral_edge, pdf_grid)

        A_list.append(A)

    A_list = np.concatenate(A_list, axis=0)


    # Method 1: naive
    # centers_naive, pdf_naive = naive_pdf_from_ipdf(R, integral_edge)

    # Method 2: tikhonov ls (nonneg)
    # logger.info(f"A:{A.shape}, R:{R.shape}")

    pdf_x = np.power(10, (pdf_grid[:-1] + pdf_grid[1:]) / 2)
    pdf_est = tikhonov_inversion(A_list, R_list_conc, lam=lam, nonneg=True, diff_order=2)
    # pdf_est = tikhonov_inversion_robust(A_list, R_list_conc, lam=lam, nonneg=True, diff_order=2)

    return pdf_x, pdf_est

    
def ipdf_to_pdf(ipdf_x, ipdf_p, pdf_x, s=0.001):
    pdf_x = np.log10(pdf_x)
    ipdf_x = np.log10(ipdf_x)
    # 排序输入数据（确保x单调增加）
    idx = np.argsort(ipdf_x)
    x = np.array(ipdf_x)[idx]
    p = np.array(ipdf_p)[idx]
    n = len(x)
    # 估计每个IPDF点对应的区间边界（取相邻中点的中间为边界）
    edges = np.empty(n+1)
    edges[1:n] = (x[:-1] + x[1:]) / 2
    # 外推两端边界
    edges[0] = x[0] - (x[1] - x[0]) / 2 if n>1 else x[0] - 0.5
    edges[n] = x[-1] + (x[-1] - x[-2]) / 2 if n>1 else x[0] + 0.5
    # 确保概率非负并归一化
    p = np.clip(p, 0, None)
    # if p.sum() > 0:
    #     p = p / p.sum()
    # 计算累积分布函数 (CDF) 在每个边界处的值
    cdf_x = edges
    cdf_y = np.concatenate(([0], np.cumsum(p)))
    # 用样条拟合CDF数据并求导得到PDF
    spl = UnivariateSpline(cdf_x, cdf_y, k=3, s=s)
    pdf_y = spl.derivative()(pdf_x)
    # 截断微小的负值并再次归一化PDF（确保概率意义）
    pdf_y[pdf_y < 0] = 0.0
    # pdf_y /= np.trapz(pdf_y, pdf_x)
    return pdf_y








#### Run

In [365]:
_drt_list = []
_nf_list = []




t0 = np.power(R1*Y1, 1/n1)


CPE_G1 = lambda x: np.sin(n1*np.pi) / (np.cosh(n1*(x-np.log(t0))) + np.cos(n1*np.pi)) /2/np.pi
CPE_G2 = lambda t: np.sin(n1*np.pi) / (np.cosh(n1*(np.log(t/t0))) + np.cos(n1*np.pi)) /2/np.pi
CPE_F1 = lambda x: np.arctan(np.tan(n1*np.pi/2) * np.tanh(n1/2*(x-np.log(t0)))) /np.pi/n1 + 0.5
CPE_F2 = lambda t: np.arctan(np.tan(n1*np.pi/2) * np.tanh(n1/2*(np.log(t/t0)))) /np.pi/n1 + 0.5

uni_order = []
uni_order_nf = []

for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _R_nf, _C_nf, _tau_nf, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z_nf[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))
    _nf_list.append(np.array([_R_nf, _C_nf, _tau_nf]))

    
    uni_order_nf.append(_tau_nf.shape[0])
    uni_order.append(_tau_i.shape[0])


if True:
    plt.figure()
    plt.loglog(_f[i,:], np.abs(_Z[i,:]), label='Simulated Data')
    # plt.plot((1/Z_sim).real, (1/Z_sim).imag, label='Simulated Data')
    plt.title(ECM_TYPE)


In [366]:



_nf_drt = []
_noise_drt = []
_theo_pdf_list = []
_theo_cdf_list = []


pdf_x = np.logspace(-np.log10(2*np.pi*f.max()), -np.log10(2*np.pi*f.min()), 501)

for i in range(_f.shape[0]):
    _drt = _drt_list[i]
    _nf = _nf_list[i]

    _R_i, _C_i, _tau_i = _drt[0,:], _drt[1,:], _drt[2,:]
    _R_nf, _C_nf, _tau_nf = _nf[0,:], _nf[1,:], _nf[2,:]


    # nf
    # pdf_x, pdf_est = Tikhonov_IPDF2PDF(_tau_nf, _R_nf, lam = 1e-3)
    # _nf_drt.append(np.array([pdf_x, pdf_est]))

    pdf_est = ipdf_to_pdf(_tau_nf, _R_nf, pdf_x, s=1)
    _nf_drt.append(np.array([pdf_x, pdf_est]))

    # noise
    # pdf_x, pdf_est = Tikhonov_IPDF2PDF(_tau_i, _R_i, lam = 1e-3)
    # _noise_drt.append(np.array([pdf_x, pdf_est]))

    pdf_est = ipdf_to_pdf(_tau_i, _R_i, pdf_x, s=1)
    _noise_drt.append(np.array([pdf_x, pdf_est]))




theo_tau_list = np.logspace(-np.log10(2*np.pi*f.max()), -np.log10(2*np.pi*f.min()), 500)

D_den_theo = np.array([CPE_G2(i) for i in theo_tau_list]) * R1
_theo_pdf_list.append(np.array([theo_tau_list, D_den_theo]))

D_den_theo = np.array([CPE_F2(i) for i in theo_tau_list]) * R1
_theo_cdf_list.append(np.array([theo_tau_list, D_den_theo]))


pdf_x_all, pdf_est_all = Tikhonov_IPDF2PDF_list(_drt_list, lam = 1e-3)

C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_33616\2163392410.py:161: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  spl = UnivariateSpline(cdf_x, cdf_y, k=3, s=s)


#### Plot

In [ ]:


# fig, axis = plt.subplots(1,1)
# axis = [axis]
# # axis = axis.flatten()
# cmap = plt.get_cmap('rainbow_r')
# for i in range(_f.shape[0]):
#     if i>4: continue
#     _R_i, _C_i, _tau_i = _drt_list[i][0,:], _drt_list[i][1,:], _drt_list[i][2,:]
#     _R_nf, _C_nf, _tau_nf = _nf_list[i][0,:], _nf_list[i][1,:], _nf_list[i][2,:]

#     _pdf_x_nf, _pdf_est_nf = _nf_drt[i][0,:], _nf_drt[i][1,:]
#     _pdf_x_drt, _pdf_est_drt = _noise_drt[i][0,:], _noise_drt[i][1,:]
#     theo_den_tau, D_den_theo = _theo_pdf_list[0][0,:], _theo_pdf_list[0][1,:]


#     axis[0].scatter(_pdf_x_nf, _pdf_est_nf, s=2, color='blue')
#     axis[0].scatter(_pdf_x_drt, _pdf_est_drt, s=2, color='red')
#     axis[0].scatter(theo_den_tau, D_den_theo, s=2, color='green')

    
#     # axis[0].scatter(_tau_i, _R_i, s=10, color='orange')
#     # axis[0].scatter(_tau_nf, _R_nf, s=10, color='orange')


# # axis[0].scatter(pdf_x_all, pdf_est_all, s=10, color='orange')

# axis[0].set_xscale('log')
# axis[0].set_yscale('log')
# # axis[0].set_aspect('equal')
# # axis[1].set_xscale('log')
# # axis[1].set_yscale('log')
# # axis[1].set_aspect('equal')









#### CDF Draft

##### Definition

In [367]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from statsmodels.nonparametric.smoothers_lowess import lowess

def ipdf_to_ipdf_loess(ipdf_x, ipdf_p, pdf_x):
    pdf_x = np.log10(pdf_x)
    ipdf_x_log = np.log10(ipdf_x)
    # 排序输入数据（确保x单调增加）
    idx = np.argsort(ipdf_x_log)
    ipdf_x_log = np.array(ipdf_x_log)[idx]
    p = np.array(ipdf_p)[idx]

    # 确保概率非负并归一化
    # p = np.clip(p, 0, None)
    p_sum = p.sum()
    if p_sum > 0:
        p = p / p_sum
    # 计算累积分布函数 (CDF) 在每个边界处的值
    p_log = np.log10(p)

    
    loess_result = lowess(p_log, ipdf_x_log, frac=0.05)
    ipdf_est = np.interp(pdf_x, loess_result[:,0], loess_result[:,1])

    pdf_x = np.power(10, pdf_x)
    ipdf_est = np.power(10, ipdf_est)

    # ipdf_est_sum = np.sum(ipdf_est)
    # p_sum = p_sum 


    return pdf_x, ipdf_est * p_sum



def ipdf_to_ipdf_gauss(ipdf_x, ipdf_p, pdf_x):
    pdf_x = np.log10(pdf_x)
    ipdf_x_log = np.log10(ipdf_x)
    # 排序输入数据（确保x单调增加）
    idx = np.argsort(ipdf_x_log)
    ipdf_x_log = np.array(ipdf_x_log)[idx]
    p = np.array(ipdf_p)[idx]

    # 确保概率非负并归一化
    # p = np.clip(p, 0, None)
    p_sum = p.sum()
    if p_sum > 0:
        p = p / p_sum
    # 计算累积分布函数 (CDF) 在每个边界处的值
    p_log = np.log10(p)

    d = np.diff(p)
    sigma_noise = 1.4826 * np.median(np.abs(d - np.median(d))) 

    
    X = ipdf_x_log.reshape(-1, 1)
    # kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=sigma_noise**2, noise_level_bounds=(1e-5, 1e-1))
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=sigma_noise**2)

    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0, normalize_y=True)
    gpr.fit(X, p_log)
    
    ipdf_est, ipdf_std = gpr.predict(pdf_x.reshape(-1,1), return_std=True)

    pdf_x = np.power(10, pdf_x)
    ipdf_est = np.power(10, ipdf_est)
    ipdf_std = np.power(10, ipdf_std)

    ipdf_est_sum = np.sum(ipdf_est)
    p_sum = p_sum / ipdf_est_sum

    logger.info(f"{ipdf_est_sum}")

    return pdf_x, ipdf_est * p_sum, ipdf_std * p_sum


def ipdf_to_ipdf(ipdf_x, ipdf_p, pdf_x):
    pdf_x = np.log10(pdf_x)
    ipdf_x_log = np.log10(ipdf_x)
    # 排序输入数据（确保x单调增加）
    idx = np.argsort(ipdf_x_log)
    ipdf_x_log = np.array(ipdf_x_log)[idx]
    p = np.array(ipdf_p)[idx]

    # 确保概率非负并归一化
    # p = np.clip(p, 0, None)
    p_sum = p.sum()
    if p_sum > 0:
        p = p / p_sum
    # 计算累积分布函数 (CDF) 在每个边界处的值
    # p = np.log10(p)

    d = np.diff(p)
    sigma_noise = 1.4826 * np.median(np.abs(d - np.median(d))) 

    logger.info(f"{sigma_noise}") # rough noise level on differences

    spl = UnivariateSpline(ipdf_x_log, p, k=3, s=sigma_noise)
    # ipdf_est = np.power(10, spl(pdf_x))
    ipdf_est = spl(pdf_x)
    pdf_x = np.power(10, pdf_x)

    
    # ipdf_est_sum = np.sum(ipdf_est)
    # p_sum = p_sum / ipdf_est_sum

    return pdf_x, ipdf_est * p_sum


def ipdf_to_cdf(ipdf_x, ipdf_p, pdf_x, s=0.001):
    pdf_x = np.log10(pdf_x)
    ipdf_x = np.log10(ipdf_x)
    # 排序输入数据（确保x单调增加）
    idx = np.argsort(ipdf_x)
    x = np.array(ipdf_x)[idx]
    p = np.array(ipdf_p)[idx]
    n = len(x)
    # 估计每个IPDF点对应的区间边界（取相邻中点的中间为边界）
    edges = np.empty(n+1)
    edges[1:n] = (x[:-1] + x[1:]) / 2
    # 外推两端边界
    edges[0] = x[0] - (x[1] - x[0]) / 2 if n>1 else x[0] - 0.5
    edges[n] = x[-1] + (x[-1] - x[-2]) / 2 if n>1 else x[0] + 0.5

    # 确保概率非负并归一化
    p = np.clip(p, 0, None)
    # p_sum = p.sum()
    # if p_sum > 0:
    #     p = p / p_sum
    # 计算累积分布函数 (CDF) 在每个边界处的值
    cdf_x = edges
    cdf_y = np.concatenate(([1e-10], np.cumsum(p)))
    cdf_y = np.log10(cdf_y)
    
    spl = UnivariateSpline(cdf_x, cdf_y, k=3, s=s)
    cdf_y = np.power(10, spl(pdf_x))
    cdf_x = np.power(10, pdf_x)

    # PDF
    pdf_y = spl.derivative()(pdf_x)
    # pdf_y = cdf_y * (np.log(10) * pdf_y)
    pdf_y = cdf_y * (pdf_y)

    return cdf_x, cdf_y, pdf_y
    # return np.power(10, cdf_x), np.power(10, cdf_y)
    # return cdf_y

## 横向对比单纯矩阵微分和加权微分，
#   PCHIP这种分段积分微分效果是最好的
from scipy.interpolate import PchipInterpolator
def ipdf_to_pdf_pchip(ipdf_x, ipdf_p, pdf_x):
    # 排序输入数据并计算CDF点（类似前述步骤）
    idx = np.argsort(ipdf_x)
    x_sorted = np.array(ipdf_x)[idx]
    p_sorted = np.clip(np.array(ipdf_p)[idx], 0, None)
    n = len(x_sorted)
    p_total = p_sorted.sum()
    if p_total == 0:
        raise ValueError("ipdf_p sum is zero; cannot form CDF.")
    p_sorted = p_sorted / p_total


    # 构建PCHIP插值器（确保extrapolate合理处理边界外点）
    # cdf_vals = np.concatenate(([1e-16], np.cumsum(p_sorted)))
    # edges = np.empty(n+1)
    # edges[1:n] = (x_sorted[:-1] + x_sorted[1:]) / 2.0
    # edges[0] = x_sorted[0] - (x_sorted[1] - x_sorted[0]) / 2 if n > 1 else x_sorted[0] - 0.5
    # edges[n] = x_sorted[-1] + (x_sorted[-1] - x_sorted[-2]) / 2 if n > 1 else x_sorted[-1] + 0.5
    # log_edges = np.log10(edges)

    # cdf_vals = np.cumsum(p_sorted)
    # edges = x_sorted
    # log_edges = np.log10(edges)

    cdf_vals = np.concatenate(([1e-16], np.cumsum(p_sorted)))
    edges = np.empty(n+1)
    x_sorted = np.log10(x_sorted)
    edges[1:n] = (x_sorted[:-1] + x_sorted[1:]) / 2.0
    edges[0] = x_sorted[0] - (x_sorted[1] - x_sorted[0]) / 2 if n > 1 else x_sorted[0] - 0.5
    edges[n] = x_sorted[-1] + (x_sorted[-1] - x_sorted[-2]) / 2 if n > 1 else x_sorted[-1] + 0.5
    log_edges = edges



    
    cdf_vals = np.log10(cdf_vals)
    pchip = PchipInterpolator(log_edges, cdf_vals, extrapolate=False)
    # 计算拟合CDF和PDF
    log_pdfx = np.log10(pdf_x)
    cdf_fitted = pchip(log_pdfx)
    # cdf_fitted = np.clip(cdf_fitted, 0.0, 1.0)
    # 计算导数并转换为PDF
    # d_cdf_dlogx = pchip.derivative()(log_pdfx)  # PCHIP返回一个PPoly，可直接求导
    # pdf_fitted = d_cdf_dlogx / (pdf_x * math.log(10))


    d_cdf_dlogx = pchip.derivative()(log_pdfx)
    # pdf_fitted = d_cdf_dlogx

    cdf_fitted = 10**cdf_fitted
    pdf_fitted = cdf_fitted * d_cdf_dlogx


    return pdf_x, cdf_fitted * p_total, pdf_fitted * p_total




def ipdf_to_pdf_weighted(ipdf_x, ipdf_p, pdf_x, s=0.001):
    # 1. 准备输入数据：转换为log10(x)，计算CDF（线性刻度）
    idx = np.argsort(ipdf_x)
    x_sorted = np.array(ipdf_x)[idx]
    p_sorted = np.clip(np.array(ipdf_p)[idx], 0, None)
    n = len(x_sorted)
    # 计算每个点对应的CDF值
    p_total = p_sorted.sum()
    if p_total == 0:
        raise ValueError("ipdf_p sum is zero; cannot form CDF.")
    p_sorted = p_sorted / p_total
    cdf_vals = np.concatenate(([0], np.cumsum(p_sorted)))  # CDF从0开始, 末项应为1
    # 构造x的边界数组（在log尺度下插值）
    edges = np.empty(n+1)
    edges[1:n] = (x_sorted[:-1] + x_sorted[1:]) / 2.0
    edges[0] = x_sorted[0] - (x_sorted[1] - x_sorted[0]) / 2 if n > 1 else x_sorted[0] - 0.5
    edges[n] = x_sorted[-1] + (x_sorted[-1] - x_sorted[-2]) / 2 if n > 1 else x_sorted[-1] + 0.5
    log_edges = np.log10(edges)
    # 2. 设置权重：尾部和两端权重较高
    w = np.ones_like(cdf_vals)
    w[0] = w[-1] = 1e3           # 确保cdf(左端)=0和cdf(右端)=1被严格拟合
    # 提高末尾若干点权重以强调尾部拟合精度（这里以最后5点为例逐步增加）
    m = min(5, len(w)-1)
    for i in range(1, m+1):
        w[-i] = max(w[-i], 100 * i)  # 尾部权重从100逐渐增加至500
    # 3. 样条拟合（log10(x) vs CDF）
    spl = UnivariateSpline(log_edges, cdf_vals, w=w, k=3, s=s)
    # 4. 计算输出CDF和PDF
    log_pdfx = np.log10(pdf_x)
    cdf_fitted = spl(log_pdfx)
    # 确保数值稳定在[0,1]范围内
    cdf_fitted = np.clip(cdf_fitted, 0.0, 1.0)
    # 样条导数 d(CDF)/d(logx)
    d_cdf_dlogx = spl.derivative()(log_pdfx)
    # 转换为 d(CDF)/dx
    # pdf_fitted = d_cdf_dlogx / (pdf_x * math.log(10))
    pdf_fitted = d_cdf_dlogx 
    return pdf_x, cdf_fitted*p_total, pdf_fitted*p_total


def dtau_distribution(tau_list):
    dtau_list   = [[0.5*(np.log10(_tau[:-1])+np.log10(_tau[1:])), 
                     np.diff(np.log10(_tau[:]))] for _tau in tau_list]
    dtau_list   = np.concatenate(dtau_list, axis=1) 

    loess_result = lowess(dtau_list[1,:], dtau_list[0,:], frac=0.1)
    loess_result = loess_result.T
    tau_dt_list = 10**loess_result
    return tau_dt_list




## 单纯的equidistribution，相较于均匀采样显著改进，
# 但是PDF抖动剧烈。
def resample_from_deltax(x_dx_array, num_points=None):
    
    # x = np.asarray(x_dx_array[0])
    # dx = np.asarray(x_dx_array[1])
    x = np.log10(x_dx_array[0])
    dx = np.log10(x_dx_array[1])
    dx = 1/dx
    

    _x_90 = np.quantile(x, [0.05, 0.95])
    _x_mask = (x >= _x_90[0]) & (x <= _x_90[1])
    x = x[_x_mask]
    dx = dx[_x_mask]

    if num_points is None:
        num_points = len(x)
    
    # 累计弧长（这里假设 Δx >= 0）
    # S = np.cumsum(dx)
    # S = np.trapezoid(dx, x)
    trapz_area = 0.5 * (dx[:-1]+dx[1:])*dx[1:]
    
    trapz_area = np.concatenate([[0.5*dx[0]*dx[0]], trapz_area])
    S = np.cumsum(trapz_area)

    S -= S[0]  # 从0开始
    
    # 在累计弧长上等间隔采样
    S_target = np.linspace(S[0], S[-1], num_points)
    
    # 反插值得到新的 x
    x_resampled = np.interp(S_target, S, x)
    
    return 10**x_resampled
    # return x_resampled

import numpy as np
from scipy.interpolate import UnivariateSpline, CubicSpline

## 好方法！相较于单纯的equidistribution，能PDF连续
def resample_equidistribution_C2(
    x_dx_array,
    n_points = None,    # 目标点数（包含两端）
    keep_frac=0.95,   # 仅在中间 keep_frac 的范围内重采样（例如 0.95 即 2.5%~97.5%）
    smooth_s=1e-2,    # 对 log m_t 的平滑强度（越大越平滑）
):

    x = np.log10(x_dx_array[0])
    dx = np.log10(x_dx_array[1])
    # x = np.log10(x_grid)
    # dx = np.log10(deltax_grid)
    dx = 1/dx
    # dx = -dx

    if n_points is None:
        n_points = len(x)    


    # _x_90 = np.quantile(x, [0.05, 0.95])
    _x_90 = np.quantile(x, [0.025, 0.975])
    _x_mask = (x >= _x_90[0]) & (x <= _x_90[1])
    x = x[_x_mask]
    dx = dx[_x_mask]


    t = x
    mt_raw = dx

    # 在 log(mt) 上做平滑样条，保证正性 & C2
    # spl_logmt = UnivariateSpline(t, np.log(mt_raw), s=smooth_s, k=3)
    spl_logmt = UnivariateSpline(t, mt_raw, s=smooth_s, k=3)
    logmt = spl_logmt(t)
    # mt = np.exp(logmt)
    mt = logmt

    # 3) 数值积分 S(t)（用梯形法在 t 上）
    S = np.zeros_like(t)
    S[1:] = np.cumsum(0.5*(mt[1:]+mt[:-1])*(t[1:]-t[:-1]))
    # 目标均匀参数（等分布）：ξ∈[0,1]，对应 S_target∈[0,S_end]
    S_end = S[-1]
    xi = np.linspace(0.0, 1.0, n_points)
    S_target = xi * S_end
    # 反插值得到离散 t_i
    t_nodes = np.interp(S_target, S, t)
    x_nodes = 10.0**t_nodes
    x_resampled = x_nodes
    return x_resampled

    # 4) 构造 C2 参数化：t(ξ) 三次样条（自然边界），x(ξ)=10^{t(ξ)}
    t_spline = CubicSpline(xi, t_nodes, bc_type='natural')  # C2
    # 给出两个实用返回：离散点 & 可评估的 C2 映射
    def x_of_xi(xi_eval):
        te = t_spline(xi_eval)         # C2
        return 10.0**te

    return x_resampled, x_of_xi



##### Run

In [374]:
plt_lines   = []
plt_labels  = []

fig, axis = plt.subplots(1,1)
axis = [axis]
# axis = axis.flatten()
cmap = plt.get_cmap('rainbow_r')


axis[0].set_xscale('log')
axis[0].set_yscale('log')
# axis[1].set_xscale('log')
# axis[1].set_yscale('log')


# R_list          = [_drt[0,1:-1] for _drt in _nf_list]
# tau_list        = [_drt[2,1:-1] for _drt in _nf_list]
R_list          = [_drt[0,1:-1] for _drt in _drt_list]
tau_list        = [_drt[2,1:-1] for _drt in _drt_list]
# R_list          = [_drt[0,:] for _drt in _drt_list]
# tau_list        = [_drt[2,:] for _drt in _drt_list]

R_list_conc     = np.concatenate(R_list, axis=0) / len(R_list)
# R_list_conc     = np.concatenate(R_list, axis=0)
tau_list_conc   = np.concatenate(tau_list, axis=0)





## Tau Discussion
tau_dt_list = dtau_distribution(tau_list)
# tau_re = resample_from_deltax(tau_dt_list, num_points=41)
# tau_re = resample_from_deltax(tau_dt_list)
tau_re = resample_equidistribution_C2(tau_dt_list)
# tau_re = resample_equidistribution_C2(tau_dt_list[0,:], tau_dt_list[1,:],n_points=tau_dt_list[0,:].shape[0])

# axis[1].scatter(tau_dt_list[0,:], tau_dt_list[1,:], s=10, color='orange')
# axis[1].scatter(tau_re[1:], np.exp(np.diff(np.log(tau_re))*100), s=10, color='red')


# pdf_x = np.logspace(math.floor(np.log10(tau_list_conc.min())), math.ceil(np.log10(tau_list_conc.max())), 501)
# pdf_x = np.logspace(np.log10(tau_list_conc.min()), np.log10(tau_list_conc.max()), 501)
pdf_x = tau_re
# pdf_x = np.logspace(-7, 0, len(tau_re))


## PDF Discussion




# from statsmodels.nonparametric.smoothers_lowess import lowess
# poi_i = 0
# for i in [0.001,0.01,0.1]:
#     R_list_conc_loess = lowess(np.log(R_list_conc), np.log(tau_list_conc), frac=i, it=3, return_sorted=False)
#     R_list_conc_loess = np.exp(R_list_conc_loess)
#     axis[0].scatter(tau_list_conc, R_list_conc_loess, s=10, color=cmap(poi_i/3))
#     poi_i = poi_i+1

# ipdf_x, ipdf_est = ipdf_to_ipdf(tau_list_conc, R_list_conc, pdf_x)
# ipdf_x, ipdf_est, _ = ipdf_to_ipdf_gauss(tau_list_conc, R_list_conc, tau_list_conc)
# ipdf_x, ipdf_est, _ = ipdf_to_ipdf_gauss(tau_list_conc, R_list_conc, pdf_x)

# ipdf_x, ipdf_est = ipdf_to_ipdf_loess(tau_list_conc, R_list_conc, tau_list_conc)
ipdf_x, ipdf_est = ipdf_to_ipdf_loess(tau_list_conc, R_list_conc, pdf_x)
# ipdf_x, ipdf_est = ipdf_to_ipdf_loess(tau_list_conc, R_list_conc, _tau_nf)

# ipdf_x, ipdf_est = tau_list_conc, R_list_conc



# axis[0].scatter(tau_list_conc, R_list_conc, s=10, color='red')
# _line1 = axis[0].scatter(ipdf_x, ipdf_est*100, s=2, color='orange')

# plt_lines.append(_line1)
# plt_labels.append(f"IPDF Equid Resample with C2 Continuity")


theo_den_tau = tau_list_conc
D_den_theo = np.array([CPE_G2(i) for i in tau_list_conc]) * R1

theo_mass_tau = tau_list_conc
D_mass_theo = np.array([CPE_F2(i) for i in tau_list_conc]) * R1







# cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_pdf_weighted(tau_list_conc, R_list_conc, pdf_x, s=0.01)
# cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_pdf_pchip(tau_list_conc, R_list_conc, pdf_x)
# cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_cdf(tau_list_conc, R_list_conc, pdf_x, s=0.1)


pdf_x = np.logspace(math.floor(np.log10(tau_list_conc.min())), math.ceil(np.log10(tau_list_conc.max())), 501)
# cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_cdf(tau_list_conc, R_list_conc, pdf_x, s=0.1)
cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_pdf_pchip(ipdf_x, ipdf_est, pdf_x)


# axis[0].scatter(cdf_nf_x, cdf_nf, s=10, color='red',marker='x')
_line2 = axis[0].scatter(cdf_nf_x, pdf_nf, s=10, color='red')

plt_lines.append(_line2)
plt_labels.append(f"PDF with Spline")








for i in range(_f.shape[0]):
    # if i!=4: continue
    _R_i, _C_i, _tau_i = _drt_list[i][0,1:-1], _drt_list[i][1,1:-1], _drt_list[i][2,1:-1]
    _R_nf, _C_nf, _tau_nf = _nf_list[i][0,1:-1], _nf_list[i][1,1:-1], _nf_list[i][2,1:-1]
    # _R_i, _C_i, _tau_i = _drt_list[i][0,:], _drt_list[i][1,:], _drt_list[i][2,:]
    # _R_nf, _C_nf, _tau_nf = _nf_list[i][0,:], _nf_list[i][1,:], _nf_list[i][2,:]

    # theo_den_tau, D_den_theo = _theo_pdf_list[0][0,:], _theo_pdf_list[0][1,:]
    # theo_mass_tau, D_mass_theo = _theo_cdf_list[0][0,:], _theo_cdf_list[0][1,:]



    # _tau_i, _R_i = ipdf_to_ipdf_loess(_tau_i, _R_i, pdf_x)
    # _tau_nf, _R_nf = ipdf_to_ipdf_loess(_tau_nf, _R_nf, _tau_nf)


    # cdf_i_x, cdf_i, pdf_i = ipdf_to_cdf(_tau_i, _R_i, pdf_x, s=1)
    # cdf_i_x, cdf_i, pdf_i = ipdf_to_pdf_pchip(_tau_i, _R_i, pdf_x)

    # cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_cdf(_tau_nf, _R_nf, pdf_x, s=0.001)
    # cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_pdf_pchip(_tau_nf, _R_nf, pdf_x)
    # cdf_nf_x, cdf_nf, pdf_nf = ipdf_to_pdf_weighted(_tau_nf, _R_nf, pdf_x, s=0.0001)


    ## CDF
    # axis[0].scatter(cdf_nf_x, cdf_nf, s=2, color='blue')
    # axis[0].scatter(cdf_i_x, cdf_i, s=2, color='red')
    # axis[0].scatter(theo_mass_tau, D_mass_theo, s=2, color='green')


    ## PDF
    # axis[0].scatter(cdf_nf_x, pdf_nf, s=2, color='blue')
    # axis[0].scatter(cdf_i_x, pdf_i, s=2, color='red')
    _line0 = axis[0].scatter(theo_den_tau, D_den_theo, s=2, color='green')

    ## Raw Data    
    # axis[0].scatter(_tau_i, _R_i, s=10, color='orange')
    # axis[0].scatter(_tau_nf, _R_nf, s=10, color='green')


    ## Tau Discussion
    # axis[1].scatter(np.sqrt(_tau_i[:-1]*_tau_i[1:]), np.exp(np.diff(np.log(_tau_i))), s=1, color='gray', label='Noise')
    # axis[1].scatter(np.sqrt(_tau_nf[:-1]*_tau_nf[1:]), np.exp(np.diff(np.log(_tau_nf))), s=10, color='blue', label='Noise Free')

    # _tau_i_re = resample_from_deltax([_tau_i[1:], np.exp(np.diff(np.log(_tau_i)))])
    # axis[0].scatter(_tau_i_re[1:], np.exp(np.diff(np.log(_tau_i_re))), s=10, color='green', label='Noise Free')

# axis[0].scatter(pdf_x_all, pdf_est_all, s=10, color='orange')




# axis[0].set_xscale('log')
# axis[0].set_yscale('log')
# axis[0].set_aspect('equal')
# axis[1].set_xscale('log')
# axis[1].set_yscale('log')
# axis[1].set_aspect('equal')


# axis[0].legend()


plt_lines.append(_line0)
plt_labels.append(f"Exact")

axis[0].legend(plt_lines, plt_labels)



#### Loewner Validation

In [148]:


for i in range(_f.shape[0]):
    if i != 10: continue
    L, Ls, H_left, H_right = Loewner_Framework(_f[i,:], _Z[i,:], REALFLAG=True)
    Ek, Ak, Bk, Ck = state_space_model(L, Ls, H_left, H_right)
    R_i, C_i, tau_i = DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG=False)
    R_real, C_real, tau_real = DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG=True)
    # _drt_list.append(np.array([_R_i, _C_i, _tau_i]))

    
fig, axis = plt.subplots(1,2)
axis = axis.flatten()

axis[0].scatter(tau_i[R_i.real>0], R_i[R_i.real>0].real, s=20, color='red')
axis[0].scatter(tau_i[R_i.real<0], -R_i[R_i.real<0].real, s=20,color='blue')
# axis[0].scatter(np.abs(tau_i), np.abs(R_i), s=5,color='green')
axis[0].scatter(tau_real, R_real , s=10, color='orange')

axis[1].scatter(tau_i[R_i.imag>0], R_i[R_i.imag>0].imag, s=20, color='red')
axis[1].scatter(tau_i[R_i.imag<0], -R_i[R_i.imag<0].imag, s=20, color='blue')
axis[1].scatter(tau_real, R_real.imag+1e-10 , s=10, color='orange')

axis[0].set_xscale('log')
axis[0].set_yscale('log')
axis[1].set_xscale('log')
axis[1].set_yscale('log')
    

In [149]:


real_mask = (tau_i.real>0)
tau_i_re = np.real(tau_i[real_mask])
R_i_re = np.real(R_i[real_mask])

# Auto Sorted bu unique               
_, _idx, _cnt = np.unique(tau_i_re, return_index=True, return_counts=True)

tau_i_re = tau_i_re[_idx]
R_i_re = R_i_re[_idx] * _cnt

tau_i_re = tau_i_re[_cnt==1]


fig, axis = plt.subplots(1,1)
# axis = axis.flatten()
axis = [axis]

axis[0].scatter(tau_i.real,tau_i.imag, s=20, color='blue')
axis[0].scatter(tau_i_re.real,tau_i_re.imag, s=5, color='red')
axis[0].set_xscale('log')
# axis[0].set_yscale('log')



# Draft

### Tikhonov

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import lsq_linear, nnls
from scipy.interpolate import PchipInterpolator
from scipy.linalg import toeplitz

# -----------------------------
# Helpers: build integration matrix A
# -----------------------------
def build_A(bin_edges, grid):
    """
    构造 A 矩阵：每个测量区间（由 bin_edges 给出）对 grid 中每个小单元的积分权重。
    bin_edges: shape (M+1,) 测量区间边界，测量 i 对应 [bin_edges[i], bin_edges[i+1]]
    grid: shape (N+1,) 细分网格的边界（通常可以等于 bin_edges 或更细）
    返回 A shape (M, N) ，p_j 定义在 grid 单元上（中心或cell-average）
    """
    M = len(bin_edges)-1
    N = len(grid)-1
    A = np.zeros((M, N))
    # 对每个测量区间 i，对每个细网格单元 j 计算重叠长度
    for i in range(M):
        L, R = bin_edges[i], bin_edges[i+1]
        # for each cell j
        for j in range(N):
            a, b = grid[j], grid[j+1]
            overlap = max(0.0, min(R, b) - max(L, a))
            A[i, j] = overlap
    return A

# -----------------------------
# Method 1: naive IPDF / Δx
# -----------------------------
def naive_pdf_from_ipdf(ipdf, bin_edges):
    widths = np.diff(bin_edges)
    pdf = ipdf / widths
    grid_centers = 0.5*(bin_edges[:-1] + bin_edges[1:])
    return grid_centers, pdf

# -----------------------------
# Method 2: Tikhonov (regularized LS), optionally non-negative
# -----------------------------
def tikhonov_inversion(A, y, lam=1e-3, nonneg=True, diff_order=2):
    """
    Solve min ||A p - y||^2 + lam ||L p||^2
    If nonneg: enforce p >= 0 via lsq_linear
    diff_order: 0 -> identity, 1 -> first diff, 2 -> second diff
    """
    N = A.shape[1]
    # build L
    if diff_order == 0:
        L = np.eye(N)
    elif diff_order == 1:
        L = np.eye(N, k=0) - np.eye(N, k=1)
        L = L[:-1]  # shape (N-1, N)
    elif diff_order == 2:
        L = np.zeros((N-2, N))
        for i in range(N-2):
            L[i, i] = 1
            L[i, i+1] = -2
            L[i, i+2] = 1
    else:
        raise ValueError("diff_order 0/1/2 only")
    # Augmented system trick: [A; sqrt(lam)*L] p = [y; 0]
    A_aug = np.vstack([A, np.sqrt(lam)*L])
    y_aug = np.concatenate([y, np.zeros(L.shape[0])])
    if nonneg:
        res = lsq_linear(A_aug, y_aug, bounds=(0, np.inf), lsmr_tol='auto', max_iter=2000)
        p = res.x
    else:
        # analytical solution via normal eqn (A^T A + lam L^T L) p = A^T y
        ATA = A.T @ A
        LTL = L.T @ L
        mat = ATA + lam * LTL
        rhs = A.T @ y
        p = np.linalg.solve(mat, rhs)
    return p

# -----------------------------
# Method 3: CDF-based (interpolate CDF, then differentiate)
# -----------------------------
def cdf_from_ipdf(ipdf, bin_edges):
    """
    ipdf: integrated pdf over each bin (i.e., integral value)
    bin_edges: edges len M+1
    返回 CDF at bin edges (len M+1)
    """
    # cumulative: CDF at right edge k = sum_{i<=k-1} ipdf[i]
    cdf = np.zeros(len(bin_edges))
    cdf[1:] = np.cumsum(ipdf)
    return cdf

def pdf_from_cdf_interp(bin_edges, ipdf, num_out_pts_per_bin=10, monotone=True):
    cdf = cdf_from_ipdf(ipdf, bin_edges)
    # CDF defined at edges (x_0 ... x_M)
    xs = bin_edges
    ys = cdf
    # use monotone (PCHIP) to avoid oscillation
    interp = PchipInterpolator(xs, ys) if monotone else PchipInterpolator(xs, ys)  # PCHIP monotone-like
    x_dense = np.linspace(xs[0], xs[-1], (len(xs)-1)*num_out_pts_per_bin)
    cdf_dense = interp(x_dense)
    # derivative (numerical)
    pdf_dense = np.gradient(cdf_dense, x_dense)
    return x_dense, pdf_dense, interp

# -----------------------------
# Bootstrap error estimate
# -----------------------------
def bootstrap_pdf(A, ipdf, solve_func, nboot=200, noise_scale=None, **solve_kwargs):
    """
    ipdf: observed integrated values (shape M)
    noise_scale: if None, estimate from residuals; else used as std dev
    solve_func: function(A, y, **solve_kwargs) -> p
    returns mean_p, std_p
    """
    M = len(ipdf)
    # estimate residual std if not provided
    if noise_scale is None:
        # simple estimate: small constant fraction
        noise_scale = max(1e-6, 0.01 * np.median(np.abs(ipdf)))
    ps = []
    for _ in range(nboot):
        yb = ipdf + np.random.normal(0, noise_scale, size=M)
        p = solve_func(A, yb, **solve_kwargs)
        ps.append(p)
    ps = np.array(ps)
    return ps.mean(axis=0), ps.std(axis=0)

# 构造样例：真实 PDF（smooth）
xgrid = np.linspace(0, 10, 501)  # fine grid boundaries
# true pdf (normalized)
xc = (xgrid[:-1] + xgrid[1:]) / 2
true_pdf = np.exp(-0.5*((xc-5)/0.6)**2)
true_pdf /= np.trapezoid(true_pdf, xc)  # normalize

# define measurement bins (coarser)
bin_edges = np.linspace(0, 10, 41)  # 40 bins
bc = (bin_edges[:-1] + bin_edges[1:]) / 2
# generate IPDF = integral over bins
A = build_A(bin_edges, xgrid)
ipdf = A @ true_pdf  # exact integrals
# add noise
ipdf_noise = ipdf + 0.01 * np.max(ipdf) * np.random.randn(len(ipdf))


In [84]:

# Method 1: naive
centers_naive, pdf_naive = naive_pdf_from_ipdf(ipdf_noise, bin_edges)

# Method 2: tikhonov ls (nonneg)
# Build a coarser grid for p unknowns (use xgrid cell-average as unknowns)
A_fine = A.copy()
# for _ in range(100):
p_est_reg = tikhonov_inversion(A_fine, ipdf_noise, lam=1e-2, nonneg=True, diff_order=2)

# Method 3: cdf interp
x_dense, pdf_dense, interp_cdf = pdf_from_cdf_interp(bin_edges, ipdf_noise, num_out_pts_per_bin=5)


In [86]:

# Bootstrap error for method 2
mean_p, std_p = bootstrap_pdf(A_fine, ipdf_noise, tikhonov_inversion, nboot=200, lam=1e-2, nonneg=True, diff_order=2)


In [85]:

# plotting
plt.figure(figsize=(10,6))
plt.plot(xc, true_pdf, label='true pdf', lw=2)
# plt.plot(centers_naive, pdf_naive, 'x-', label='naive ipdf/Δx')
plt.plot(xc, p_est_reg, '-', label='tikhonov reg (est)', lw=2)
plt.fill_between(xc, mean_p-std_p, mean_p+std_p, color='gray', alpha=0.8, label='bootstrap ±1σ (reg)')
# plt.plot(x_dense, pdf_dense, '--', label='from CDF interp')


# plt.plot(bc, ipdf, label='true pdf', lw=2)
# plt.plot(bc, ipdf_noise, label='true pdf', lw=2)

plt.legend()
plt.xlabel('x')
plt.ylabel('pdf')
plt.title('Reconstruction from IPDF (demo)')
plt.show()


### Inversion

In [42]:
# save as ipdf_to_pdf_examples.py
import numpy as np
from scipy import optimize, integrate, special
from scipy.optimize import lsq_linear
import matplotlib.pyplot as plt

# ---------- helper: build Δx from sample points (centers) ----------
def estimate_bin_edges_from_centers(x):
    x = np.asarray(x)
    order = np.argsort(x)
    xs = x[order]
    # internal edges: midpoints
    mid = 0.5*(xs[1:] + xs[:-1])
    edges = np.empty(len(xs)+1)
    edges[1:-1] = mid
    edges[0] = xs[0] - (mid[0]-xs[0])
    edges[-1]= xs[-1] + (xs[-1]-mid[-1])
    widths = edges[1:] - edges[:-1]
    return edges, widths, order

# ---------- synthesize data: true gaussian, compute IPDF (integrals) ----------
def synth_ipdf_from_pdf(pdf_fun, edges):
    # edges: bin edges array length n+1
    b = np.array([integrate.quad(pdf_fun, edges[i], edges[i+1])[0] for i in range(len(edges)-1)])
    return b

# ---------- Method A: parametric Gaussian fit via minimizing bin-integral squared error ----------
def fit_gaussian_by_bin_integrals(x_centers, ipdf, edges):
    # model: normal pdf with params (mu, sigma)
    def model_bin_integrals(params):
        mu, log_sigma = params
        sigma = np.exp(log_sigma)
        cdf = lambda z: 0.5*(1 + special.erf((z - mu)/(sigma*np.sqrt(2))))
        ints = np.array([cdf(edges[i+1]) - cdf(edges[i]) for i in range(len(edges)-1)])
        return ints
    def loss(params):
        ints = model_bin_integrals(params)
        return np.sum((ints - ipdf)**2)
    # init: mean ~ weighted center, sigma ~ std estimate
    mu0 = (x_centers * ipdf).sum() / ipdf.sum()
    sigma0 = max(1e-3, np.sqrt(np.abs(( (x_centers-mu0)**2 * ipdf ).sum() / ipdf.sum())))
    res = optimize.minimize(loss, x0=[mu0, np.log(sigma0)], method='L-BFGS-B')
    mu_hat, sigma_hat = res.x[0], np.exp(res.x[1])
    return mu_hat, sigma_hat, res

# ---------- Method B: linear inversion with Tikhonov (discrete grid) ----------
def tikhonov_inversion(x_grid, x_centers, ipdf, edges, lam=1e-2, order=2):
    # Discretize PDF on x_grid; build A such that ipdf ~= A @ p * dx
    dx = np.diff(x_grid)
    nx = len(x_grid)
    m = len(ipdf)
    A = np.zeros((m, nx))
    # for each bin, mark grid points whose centers lie inside bin, use trapezoid weighting
    for i in range(m):
        left, right = edges[i], edges[i+1]
        # compute overlap of each grid cell [xj - dxj/2, xj + dxj/2] with [left,right]
        cell_left = x_grid - 0.5*np.concatenate(([dx[0]], dx))[:nx]  # approximate
        cell_right = x_grid + 0.5*np.concatenate((dx, [dx[-1]]))[:nx]
        overlap = np.maximum(0, np.minimum(cell_right, right) - np.maximum(cell_left, left))
        A[i,:] = overlap  # so ipdf_i ~= sum_j overlap_j * p_j
    # regularization matrix D (finite difference)
    if order==2:
        D = np.zeros((nx-2, nx))
        for i in range(nx-2):
            D[i, i] = 1
            D[i, i+1] = -2
            D[i, i+2] = 1
    else:
        D = np.eye(nx)
    # augment system: [A; sqrt(lam)*D] p = [ipdf; 0]
    sqrtlam = np.sqrt(lam)
    A_aug = np.vstack([A, sqrtlam * D])
    b_aug = np.concatenate([ipdf, np.zeros(D.shape[0])])
    # bounds: nonnegativity
    res = lsq_linear(A_aug, b_aug, bounds=(0, np.inf), lsmr_tol='auto', max_iter=2000)
    p = res.x
    # convert p from "per-grid-cell integral" to density: divide by cell widths
    # But our A already used overlaps, so p approximates density values; more robust: normalize
    return p, res

# ---------- Method C: Richardson-Lucy style for linear operator A ----------
def richardson_lucy_matrix(A, b, iters=100, eps=1e-12):
    # A: m x n, b: m
    m, n = A.shape
    p = np.ones(n)
    At1 = A.sum(axis=0)  # A^T 1
    for k in range(iters):
        Ap = A.dot(p) + eps
        ratio = b / Ap
        update = A.T.dot(ratio)
        p = p * (update / (At1 + eps))
        # optional: enforce nonnegativity trivially by clipping
        p = np.maximum(p, 0)
    return p


In [ ]:

np.random.seed(0)
# 1) define true pdf (gaussian)
mu_true = 0.5; sigma_true = 0.2
pdf_true = lambda x: (1/(sigma_true*np.sqrt(2*np.pi)))*np.exp(-0.5*((x-mu_true)/sigma_true)**2)
# build a fine grid for true pdf visualization
xgrid_fine = np.linspace(-0.5, 1.5, 2000)
pdf_vals = pdf_true(xgrid_fine)
# 2) sample non-uniform centers and estimate edges
centers = np.sort(np.concatenate([np.linspace(-0.4,0.2,10), np.linspace(0.25,1.4,20)]))
edges, widths, order = estimate_bin_edges_from_centers(centers)
# 3) synthesize IPDF by integrating true pdf over edges
b = synth_ipdf_from_pdf(pdf_true, edges)
# add small noise
noise_level = 1e-1
b_noisy = b + np.random.normal(0, noise_level, b.shape) * b
# b_noisy = np.clip(b_noisy, 0, None)

# Method A: parametric gaussian fit
mu_hat, sigma_hat, resA = fit_gaussian_by_bin_integrals(centers, b_noisy, edges)
print("Param fit: mu_hat, sigma_hat =", mu_hat, sigma_hat)

# Method B: Tikhonov inversion on grid
x_grid = np.linspace(-0.6, 1.6, 300)
p_tik, resB = tikhonov_inversion(x_grid, centers, b_noisy, edges, lam=1e-3, order=2)

# Method C: RL using same A as in B
# rebuild A used earlier:
dx = np.diff(x_grid)
nx = len(x_grid)
m = len(b_noisy)
A = np.zeros((m, nx))
cell_left = x_grid - 0.5*np.concatenate(([dx[0]], dx))[:nx]
cell_right = x_grid + 0.5*np.concatenate((dx, [dx[-1]]))[:nx]
for i in range(m):
    left, right = edges[i], edges[i+1]
    overlap = np.maximum(0, np.minimum(cell_right, right) - np.maximum(cell_left, left))
    A[i,:] = overlap
p0 = np.ones(nx)
p_rl = richardson_lucy_matrix(A, b_noisy, iters=200)

# normalize densities for plotting
p_tik = p_tik / (np.trapezoid(p_tik, x_grid) + 1e-12)
p_rl  = p_rl  / (np.trapezoid(p_rl, x_grid) + 1e-12)
pdf_true_norm = pdf_vals / (np.trapz(pdf_vals, xgrid_fine))

# plot
plt.figure(figsize=(8,4))
plt.plot(xgrid_fine, pdf_true_norm, label='true pdf')
plt.plot(x_grid, p_tik, label='tikhonov')
plt.plot(x_grid, p_rl, label='richardson-lucy')
# param fit pdf
pdf_param = lambda x: (1/(sigma_hat*np.sqrt(2*np.pi)))*np.exp(-0.5*((x-mu_hat)/sigma_hat)**2)
plt.plot(xgrid_fine, pdf_param(xgrid_fine), '--', label='param gaussian fit')
plt.legend(); plt.title('PDF recovery examples')
plt.show()


In [57]:
plt.figure()
plt.scatter(centers, b)
plt.scatter(centers, b_noisy)
plt.yscale('log')

In [87]:
import numpy as np
from scipy.interpolate import UnivariateSpline

def ipdf_to_pdf(ipdf_x, ipdf_p, pdf_x, s=0.001):
    # 排序输入数据（确保x单调增加）
    idx = np.argsort(ipdf_x)
    x = np.array(ipdf_x)[idx]
    p = np.array(ipdf_p)[idx]
    n = len(x)
    # 估计每个IPDF点对应的区间边界（取相邻中点的中间为边界）
    edges = np.empty(n+1)
    edges[1:n] = (x[:-1] + x[1:]) / 2
    # 外推两端边界
    edges[0] = x[0] - (x[1] - x[0]) / 2 if n>1 else x[0] - 0.5
    edges[n] = x[-1] + (x[-1] - x[-2]) / 2 if n>1 else x[0] + 0.5
    # 确保概率非负并归一化
    p = np.clip(p, 0, None)
    # if p.sum() > 0:
    #     p = p / p.sum()
    # 计算累积分布函数 (CDF) 在每个边界处的值
    cdf_x = edges
    cdf_y = np.concatenate(([0], np.cumsum(p)))
    # 用样条拟合CDF数据并求导得到PDF
    spl = UnivariateSpline(cdf_x, cdf_y, k=3, s=s)
    pdf_y = spl.derivative()(pdf_x)
    # 截断微小的负值并再次归一化PDF（确保概率意义）
    pdf_y[pdf_y < 0] = 0.0
    # pdf_y /= np.trapz(pdf_y, pdf_x)
    return pdf_y

# 示例：对给定IPDF数据（x位置和对应概率）估计PDF
# ipdf_x = np.array([-2, -1, 0, 1, 2])            # IPDF中心点（不均匀分布）
# ipdf_p = np.array([0.13, 0.34, 0.35, 0.13, 0.05])*10  # 每个区间的概率（含噪声）
ipdf_x = np.log10(tau_ii)         # IPDF中心点（不均匀分布）
ipdf_p = R_ii  # 每个区间的概率（含噪声）
pdf_x = np.linspace(math.floor(np.min(ipdf_x)), math.ceil(np.max(ipdf_x)), 51)      # 希望输出PDF的x网格（均匀）
pdf_est = ipdf_to_pdf(ipdf_x, ipdf_p, pdf_x)

plt.figure()
plt.scatter(pdf_x, pdf_est)
plt.scatter(ipdf_x, ipdf_p)

In [70]:
R_ii = _R_i
tau_ii = _tau_i

## CDF + Logistic

In [210]:
import numpy as np
from scipy.interpolate import UnivariateSpline

def _centers_to_edges(x):
    """把区间中心点 x 转成边界 edges（两端用相邻间距外推）。"""
    x = np.asarray(x)
    idx = np.argsort(x)
    x = x[idx]
    n = len(x)
    edges = np.empty(n + 1, dtype=float)
    if n == 1:  # 退化情形
        h = 0.5
        edges[0], edges[1] = x[0] - h, x[0] + h
        return edges
    edges[1:n] = 0.5 * (x[:-1] + x[1:])
    edges[0]   = x[0] - 0.5 * (x[1] - x[0])
    edges[n]   = x[-1] + 0.5 * (x[-1] - x[-2])
    return edges

def _logit(u, eps=1e-9):
    u = np.clip(u, eps, 1.0 - eps)
    return np.log(u) - np.log(1.0 - u)

def _sigmoid(z):
    # 数值稳定版 sigmoid
    out = np.empty_like(z, dtype=float)
    pos = z >= 0
    out[pos]  = 1.0 / (1.0 + np.exp(-z[pos]))
    ez = np.exp(z[~pos])
    out[~pos] = ez / (1.0 + ez)
    return out

def _cdf_from_ipdf(ipdf_p):
    """由区间概率 p 得到边界上的 CDF（左->右）。"""
    p = np.asarray(ipdf_p, dtype=float)
    p = np.clip(p, 0, None)
    if p.sum() > 0:
        p = p / p.sum()
    F_edges = np.concatenate(([0.0], np.cumsum(p)))
    return F_edges

def _right_cumulative(ipdf_p):
    """右向累积（生存函数 S 的“右->左”累加）；返回与 edges 对齐的 1 - F_R。"""
    p = np.asarray(ipdf_p, dtype=float)
    p = np.clip(p, 0, None)
    if p.sum() > 0:
        p = p / p.sum()
    # 从右往左：S_k = sum_{i>=k} p_i, 于是 F_R = 1 - S
    S_edges_right2left = np.concatenate((np.cumsum(p[::-1])[::-1], [0.0]))  # 与左边界对齐
    F_from_right = 1.0 - S_edges_right2left
    return F_from_right

def _default_cdf_weights(n_edges):
    """
    对 CDF 点做简单方差加权：w_k ~ 1/sqrt(k+1) ，
    反映左->右累加时误差方差递增的趋势（不知道真实方差时的保守缺省）。
    """
    k = np.arange(n_edges, dtype=float)  # 0..n
    w = 1.0 / np.sqrt(k + 1.0)
    # 两端稍微减弱一点，防止边界“拉扯”
    w[0] *= 0.7
    w[-1] *= 0.7
    return w

def _fit_logit_cdf_and_diff(edges, F_edges, s=0.0, w=None):
    """
    在 logit(F) 上做样条拟合，返回在任意 x 上评估 PDF 的闭包：
      f(x) = sigmoid(g(x)) * (1 - sigmoid(g(x))) * g'(x)
    """
    if w is None:
        w = _default_cdf_weights(len(edges))
    g_vals = _logit(F_edges)
    spl = UnivariateSpline(edges, g_vals, w=w, s=s, k=3)
    def pdf_eval(x):
        x = np.asarray(x, dtype=float)
        g = spl(x)
        gp = spl.derivative()(x)
        sig = _sigmoid(g)
        f = sig * (1.0 - sig) * gp
        return f
    return pdf_eval

def pdf_from_ipdf_logit_two_sided(ipdf_x, ipdf_p, pdf_x, s=0.0, blend_width=0.2, eps=1e-12):
    """
    方案A：双向累积 + logit(CDF) 样条 + 链式法则求导，并在中部平滑融合左右两个 PDF 估计。
    参数：
      ipdf_x: 区间中心点（不必均匀）
      ipdf_p: 对应每个区间的概率（可带噪声，和不要求正好为1）
      pdf_x : 需要输出 PDF 的均匀网格
      s     : 样条平滑参数（越大越平滑；可用CV/网格搜索）
      blend_width: 融合带宽（占整体区间长度的比例，0.1~0.3 常用）
      eps   : 数值稳定项
    返回：
      pdf_y : 在 pdf_x 上的 PDF 估计（>=0，已归一化）
    """
    ipdf_x = np.asarray(ipdf_x, dtype=float)
    ipdf_p = np.asarray(ipdf_p, dtype=float)
    pdf_x  = np.asarray(pdf_x, dtype=float)

    # 1) 构造边界
    edges = _centers_to_edges(ipdf_x)

    # 2) 左累积的 CDF，并在 logit 空间拟合
    F_L = _cdf_from_ipdf(ipdf_p)
    wL  = _default_cdf_weights(len(edges))
    pdf_L = _fit_logit_cdf_and_diff(edges, F_L, s=s, w=wL)

    # 3) 右累积（通过 F_R=1-S），同样拟合
    F_R = _right_cumulative(ipdf_p)
    wR  = wL[::-1]  # 右侧也做相反方向的方差加权
    pdf_R = _fit_logit_cdf_and_diff(edges, F_R, s=s, w=wR)

    fL = np.maximum(pdf_L(pdf_x), 0.0)
    fR = np.maximum(pdf_R(pdf_x), 0.0)

    # 4) 在中部平滑融合：靠左更信任 fL，靠右更信任 fR
    x_left, x_right = edges[0], edges[-1]
    xm = 0.5 * (x_left + x_right)
    width = blend_width * (x_right - x_left)
    # logistic 融合权
    alpha = 1.0 / (1.0 + np.exp((pdf_x - xm) / (width + 1e-12)))

    f = alpha * fL + (1.0 - alpha) * fR

    # 5) 数值修正与归一化
    f = np.clip(f, 0.0, None)
    area = np.trapz(f, pdf_x)
    if area > 0:
        f = f / area
    else:
        # 极端失败时（几乎不发生），退回均匀
        dx = np.mean(np.diff(pdf_x))
        f = np.ones_like(pdf_x) / (len(pdf_x) * dx)
    return f


In [213]:
# 例：带噪声的“钟形”IPDF（以高斯为例）
rng = np.random.default_rng(0)
true_pdf = lambda x: np.exp(-0.5*x**2)/np.sqrt(2*np.pi)
# 构造不均匀区间中心与区间概率（加噪声）
ipdf_x = np.array([-3.0, -1.8, -1.2, -0.6, -0.2, 0.2, 0.6, 1.0, 1.6, 2.6])
edges = _centers_to_edges(ipdf_x)
# 理论区间概率
from scipy.stats import norm
p_clean = norm.cdf(edges[1:]) - norm.cdf(edges[:-1])
# 加噪声（正负都有，随后会归一化）
noise = 0.05 * rng.normal(size=p_clean.size)  # 5% 噪声
ipdf_p = np.clip(p_clean + noise, 0, None)
if ipdf_p.sum() > 0:
    ipdf_p /= ipdf_p.sum()

pdf_x = np.linspace(edges[0], edges[-1], 501)
pdf_est = pdf_from_ipdf_logit_two_sided(ipdf_x, ipdf_p, pdf_x, s=1e-3, blend_width=0.2)


plt.figure()
plt.scatter(pdf_x, pdf_est, color = 'red')
plt.scatter(pdf_x, true_pdf(pdf_x), color = 'blue')


C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_4868\734958049.py:127: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(f, pdf_x)


## CDF Modified

In [220]:
import numpy as np
from scipy.interpolate import PchipInterpolator

def _centers_to_edges(x):
    x = np.asarray(x, float)
    idx = np.argsort(x); x = x[idx]
    n = x.size
    edges = np.empty(n+1, float)
    if n == 1:
        h = 0.5
        edges[:] = (x[0]-h, x[0]+h)
        return edges
    edges[1:-1] = 0.5*(x[:-1] + x[1:])
    edges[0]    = x[0] - 0.5*(x[1] - x[0])
    edges[-1]   = x[-1] + 0.5*(x[-1] - x[-2])
    return edges

def _logit(u, eps=1e-9):
    u = np.clip(u, eps, 1.0-eps)
    return np.log(u) - np.log(1.0 - u)

def _sigmoid(z):
    out = np.empty_like(z, float)
    pos = z >= 0
    out[pos]  = 1.0/(1.0 + np.exp(-z[pos]))
    ez = np.exp(z[~pos])
    out[~pos] = ez/(1.0 + ez)
    return out

def _cdf_from_ipdf(p):
    p = np.asarray(p, float)
    p = np.clip(p, 0, None)
    s = p.sum()
    if s > 0: p = p/s
    return np.concatenate(([0.0], np.cumsum(p)))

def _cdf_from_right(p):
    p = np.asarray(p, float)
    p = np.clip(p, 0, None)
    s = p.sum()
    if s > 0: p = p/s
    # 右向累积的生存函数 S：S_k = sum_{i>=k} p_i
    S_edges = np.concatenate((np.cumsum(p[::-1])[::-1], [0.0]))
    F_from_right = 1.0 - S_edges
    return F_from_right

def _fit_logit_cdf_pchip(edges, F_edges):
    """
    在 g=logit(F) 上做 PCHIP（单调、无超调），返回可评估 PDF 的闭包：
      f(x) = sigma(g)*(1-sigma(g)) * g'(x)
    """
    g = _logit(F_edges)
    # PCHIP 对单调的 g 会给出单调插值；g'(x) >= 0
    g_spline = PchipInterpolator(edges, g, extrapolate=True)
    def pdf_eval(x):
        x = np.asarray(x, float)
        g_val = g_spline(x)
        gp    = g_spline.derivative()(x)
        sig   = _sigmoid(g_val)
        f     = sig*(1.0 - sig)*gp
        return f
    return pdf_eval

def pdf_from_ipdf_logit_pchip_two_sided(ipdf_x, ipdf_p, pdf_x, blend_width=0.2):
    """
    双向累积 + logit(CDF) + PCHIP + 融合（单组 IPDF 版本）
    - ipdf_x: 区间中心（可不均匀）
    - ipdf_p: 该组区间概率（可带噪声）
    - pdf_x : 输出网格（一般均匀）
    - blend_width: 左右 PDF 融合的过渡带（占区间长度比例）
    """
    ipdf_x = np.asarray(ipdf_x, float)
    ipdf_p = np.asarray(ipdf_p, float)
    pdf_x  = np.asarray(pdf_x,  float)

    edges = _centers_to_edges(ipdf_x)

    # 左累积 & 右累积
    F_L = _cdf_from_ipdf(ipdf_p)
    F_R = _cdf_from_right(ipdf_p)

    # 在 logit(CDF) 上用 PCHIP，保证单调且抑制超调
    pdf_L = _fit_logit_cdf_pchip(edges, F_L)
    pdf_R = _fit_logit_cdf_pchip(edges, F_R)

    fL = np.maximum(pdf_L(pdf_x), 0.0)
    fR = np.maximum(pdf_R(pdf_x), 0.0)

    # 平滑融合：靠左更信任 fL，靠右更信任 fR
    xL, xR = edges[0], edges[-1]
    xm = 0.5*(xL + xR)
    width = max(1e-12, blend_width * (xR - xL))
    alpha = 1.0/(1.0 + np.exp((pdf_x - xm)/width))
    f = alpha*fL + (1.0 - alpha)*fR

    # 归一化 & 裁负
    f = np.clip(f, 0.0, None)
    area = np.trapz(f, pdf_x)
    if area > 0:
        f /= area
    return f


In [223]:
# 例：带噪声的“钟形”IPDF（以高斯为例）
rng = np.random.default_rng(0)
true_pdf = lambda x: np.exp(-0.5*x**2)/np.sqrt(2*np.pi)
# 构造不均匀区间中心与区间概率（加噪声）
ipdf_x = np.array([-3.0, -1.8, -1.2, -0.6, -0.2, 0.2, 0.6, 1.0, 1.6, 2.6])
edges = _centers_to_edges(ipdf_x)
# 理论区间概率
from scipy.stats import norm
p_clean = norm.cdf(edges[1:]) - norm.cdf(edges[:-1])
# 加噪声（正负都有，随后会归一化）
noise = 0.05 * rng.normal(size=p_clean.size)  # 5% 噪声
ipdf_p = np.clip(p_clean + noise, 0, None)
if ipdf_p.sum() > 0:
    ipdf_p /= ipdf_p.sum()

pdf_x = np.linspace(edges[0], edges[-1], 501)
pdf_est = pdf_from_ipdf_logit_pchip_two_sided(ipdf_x, ipdf_p, pdf_x, blend_width=0.2)


plt.figure()
plt.scatter(pdf_x, pdf_est, color = 'red')
plt.scatter(pdf_x, true_pdf(pdf_x), color = 'blue')


C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_4868\731505302.py:98: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  area = np.trapz(f, pdf_x)
